In [1]:
"""
======================================================================================
BƯỚC 1: CLEAN VÀ TIỀN XỬ LÝ DATASET CICIDS2018 CHO MÔ HÌNH CNN
======================================================================================

Script này thực hiện các bước tiền xử lý dữ liệu:
1. Đọc từng file CSV theo chunks để tối ưu bộ nhớ
2. Loại bỏ các cột không cần thiết (IP, Port, Timestamp, Flow ID)
3. Loại bỏ các cột có variance = 0 (zero-variance columns)
4. Xử lý Infinity và NaN bằng Mode của cột
5. Loại bỏ các hàng trùng lặp
6. Chuyển đổi nhãn sang dạng binary (Benign=0, Attack=1)
7. Lưu dữ liệu đã clean vào folder để sử dụng sau

Có thể chạy trên cả Kaggle và Local
"""

import os
import numpy as np
import pandas as pd
import pickle
import json
import gc
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra môi trường chạy (Kaggle hoặc Local)
IS_KAGGLE = os.path.exists('/kaggle/input')

# Progress bar
try:
    from tqdm import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("⚠️  tqdm không có sẵn. Cài đặt bằng: pip install tqdm")
    tqdm = lambda x, **kwargs: x

# ============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================================
if IS_KAGGLE:
    # Đường dẫn trên Kaggle - thay đổi theo dataset của bạn
    DATA_DIR = "/kaggle/input/cicids2018"  # Thay đổi nếu tên dataset khác
    OUTPUT_DIR = "/kaggle/working/cleaned_data"
    print("🌐 Đang chạy trên KAGGLE")
else:
    # Đường dẫn Local
    DATA_DIR = r"D:\PROJECT\Machine Learning\IOT\CICIDS2018-CSV"
    OUTPUT_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\cleaned_data"
    print("💻 Đang chạy trên LOCAL")

# ============================================================================
# CẤU HÌNH XỬ LÝ DỮ LIỆU
# ============================================================================

# Kích thước chunk khi đọc CSV (điều chỉnh theo RAM của máy)
CHUNK_SIZE = 300000  # 300k rows mỗi chunk

# Random state để tái tạo kết quả
RANDOM_STATE = 42

# ============================================================================
# DANH SÁCH CÁC CỘT CẦN LOẠI BỎ (Identification columns)
# ============================================================================

COLUMNS_TO_DROP = [
    'Flow ID',          # ID duy nhất cho mỗi flow - không có ý nghĩa phân loại
    'Src IP',           # IP nguồn - không tổng quát
    'Dst IP',           # IP đích - không tổng quát
    'Src Port',         # Port nguồn - có thể bị overfitting
    'Dst Port',         # Port đích - có thể bị overfitting
    'Timestamp',        # Thời gian - không liên quan đến pattern tấn công
]

# Cột nhãn
LABEL_COLUMN = 'Label'

# ============================================================================
# CLASS XỬ LÝ DỮ LIỆU
# ============================================================================

class CICIDS2018_DataCleaner:
    """
    Class clean dữ liệu CICIDS2018 cho mô hình CNN

    Các bước xử lý:
    1. Đọc dữ liệu theo chunks
    2. Loại bỏ cột identification
    3. Loại bỏ zero-variance columns
    4. Xử lý Infinity và NaN bằng Mode
    5. Loại bỏ duplicate
    6. Chuyển đổi nhãn sang binary
    7. Lưu dữ liệu đã clean
    """

    def __init__(self, data_dir, output_dir, chunk_size=CHUNK_SIZE):
        """
        Khởi tạo data cleaner

        Args:
            data_dir: Đường dẫn thư mục chứa file CSV
            output_dir: Đường dẫn thư mục lưu kết quả
            chunk_size: Số dòng mỗi chunk khi đọc CSV
        """
        self.data_dir = Path(data_dir)
        self.output_dir = Path(output_dir)
        self.chunk_size = chunk_size

        # Tạo thư mục output nếu chưa có
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # Lưu tên các features và thông tin xử lý
        self.feature_names = None
        self.zero_variance_cols = []
        self.column_modes = {}  # Lưu mode của từng cột để xử lý NaN/Inf

        # Thống kê
        self.stats = {
            'total_rows_read': 0,
            'rows_after_cleaning': 0,
            'duplicates_removed': 0,
            'nan_replaced': 0,
            'inf_replaced': 0,
            'zero_variance_cols_removed': 0,
            'benign_count': 0,
            'attack_count': 0,
            'feature_count': 0,
            'processing_time': 0.0  # Float để lưu thời gian xử lý (giây)
        }

    def _get_csv_files(self):
        """Lấy danh sách các file CSV trong thư mục data"""
        csv_files = list(self.data_dir.glob("*_TrafficForML_CICFlowMeter.csv"))
        if not csv_files:
            # Thử pattern khác cho Kaggle
            csv_files = list(self.data_dir.glob("*.csv"))
            # Loại bỏ file zip nếu có
            csv_files = [f for f in csv_files if not f.name.endswith('.zip')]

        if not csv_files:
            raise FileNotFoundError(f"Không tìm thấy file CSV trong {self.data_dir}")

        print(f"\n📂 Tìm thấy {len(csv_files)} file CSV:")
        for f in sorted(csv_files):
            print(f"   - {f.name}")
        return sorted(csv_files)

    def _clean_column_names(self, df):
        """Chuẩn hóa tên cột (loại bỏ khoảng trắng thừa)"""
        df.columns = df.columns.str.strip()
        return df

    def _drop_identification_columns(self, df):
        """Loại bỏ các cột identification không cần thiết cho huấn luyện"""
        columns_to_drop = [col for col in COLUMNS_TO_DROP if col in df.columns]
        if columns_to_drop:
            df = df.drop(columns=columns_to_drop)
        return df

    def _convert_to_numeric(self, df):
        """Chuyển đổi các cột về dạng số"""
        feature_cols = [col for col in df.columns if col != LABEL_COLUMN]
        for col in feature_cols:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col], errors='coerce')
        return df

    def _convert_to_binary_label(self, df):
        """
        Chuyển đổi nhãn sang dạng binary:
        - Benign -> 0 (lưu lượng bình thường)
        - Tất cả các loại tấn công khác -> 1 (lưu lượng bất thường)
        """
        if LABEL_COLUMN not in df.columns:
            raise ValueError(f"Không tìm thấy cột '{LABEL_COLUMN}' trong dữ liệu")

        # Chuẩn hóa nhãn (loại bỏ khoảng trắng, lowercase)
        df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(str).str.strip().str.lower()

        # Loại bỏ các hàng có nhãn là 'label' (header bị lẫn vào data)
        df = df[df[LABEL_COLUMN] != 'label']

        # Chuyển đổi sang binary: Benign=0, Attack=1
        df['binary_label'] = (df[LABEL_COLUMN] != 'benign').astype(int)

        # Xóa cột Label gốc, giữ lại binary_label
        df = df.drop(columns=[LABEL_COLUMN])

        return df

    def _first_pass_collect_info(self, csv_files):
        """
        Lần đọc đầu tiên: Thu thập thông tin về columns và tính mode

        Mục đích:
        - Xác định các cột có variance = 0
        - Tính mode của từng cột để thay thế NaN/Inf
        """
        print("\n" + "="*80)
        print("📊 BƯỚC 1: THU THẬP THÔNG TIN TỪ DỮ LIỆU")
        print("="*80)

        all_columns = None
        column_value_counts = {}  # Để tính mode
        column_min_max = {}  # Để kiểm tra variance

        for csv_file in csv_files:
            print(f"\n   Đang scan: {csv_file.name}")
            chunk_iterator = pd.read_csv(csv_file, chunksize=self.chunk_size,
                                        low_memory=False, encoding='utf-8')

            for chunk in chunk_iterator:
                chunk = self._clean_column_names(chunk)
                chunk = self._drop_identification_columns(chunk)
                chunk = self._convert_to_numeric(chunk)

                if all_columns is None:
                    all_columns = [col for col in chunk.columns if col != LABEL_COLUMN]
                    for col in all_columns:
                        column_value_counts[col] = {}
                        column_min_max[col] = {'min': np.inf, 'max': -np.inf}

                # Thu thập thông tin cho mỗi cột
                for col in all_columns:
                    if col in chunk.columns:
                        # Thay thế inf trước khi tính
                        col_data = chunk[col].replace([np.inf, -np.inf], np.nan)
                        valid_data = col_data.dropna()

                        if len(valid_data) > 0:
                            # Cập nhật min/max
                            col_min = valid_data.min()
                            col_max = valid_data.max()
                            column_min_max[col]['min'] = min(column_min_max[col]['min'], col_min)
                            column_min_max[col]['max'] = max(column_min_max[col]['max'], col_max)

                            # Thu thập value counts cho mode (lấy top 10 để tiết kiệm bộ nhớ)
                            vc = valid_data.value_counts().head(10).to_dict()
                            for val, count in vc.items():
                                if val not in column_value_counts[col]:
                                    column_value_counts[col][val] = 0
                                column_value_counts[col][val] += count

                gc.collect()

        # Xác định zero-variance columns
        print("\n   Đang phân tích variance của các cột...")
        for col in all_columns:
            if column_min_max[col]['min'] == column_min_max[col]['max']:
                self.zero_variance_cols.append(col)

        # Tính mode cho mỗi cột
        print("   Đang tính mode cho mỗi cột...")
        for col in all_columns:
            if col not in self.zero_variance_cols:
                if column_value_counts[col]:
                    # Mode là giá trị xuất hiện nhiều nhất
                    mode_val = max(column_value_counts[col], key=column_value_counts[col].get)
                    self.column_modes[col] = mode_val
                else:
                    self.column_modes[col] = 0  # Fallback nếu không có dữ liệu hợp lệ

        self.stats['zero_variance_cols_removed'] = len(self.zero_variance_cols)

        print(f"\n   ✅ Số cột zero-variance sẽ loại bỏ: {len(self.zero_variance_cols)}")
        if self.zero_variance_cols:
            print(f"      Các cột: {self.zero_variance_cols}")
        print(f"   ✅ Số cột sẽ giữ lại: {len(all_columns) - len(self.zero_variance_cols)}")

        return all_columns

    def _handle_nan_inf_with_mode(self, df):
        """
        Xử lý NaN và Infinity bằng Mode của cột

        Replace Infinity and NaN with the Mode of the column
        """
        feature_cols = [col for col in df.columns if col != 'binary_label']

        for col in feature_cols:
            if col in self.column_modes:
                mode_val = self.column_modes[col]

                # Đếm số lượng inf và nan
                inf_mask = np.isinf(df[col])
                nan_mask = df[col].isna()

                self.stats['inf_replaced'] += inf_mask.sum()
                self.stats['nan_replaced'] += nan_mask.sum()

                # Thay thế inf bằng nan trước
                df[col] = df[col].replace([np.inf, -np.inf], np.nan)

                # Thay thế tất cả nan bằng mode
                df[col] = df[col].fillna(mode_val)

        return df

    def _drop_zero_variance_columns(self, df):
        """Loại bỏ các cột có variance = 0"""
        cols_to_drop = [col for col in self.zero_variance_cols if col in df.columns]
        if cols_to_drop:
            df = df.drop(columns=cols_to_drop)
        return df

    def _process_single_file(self, csv_file):
        """
        Xử lý một file CSV theo chunks

        Args:
            csv_file: Đường dẫn file CSV

        Returns:
            DataFrame đã được xử lý
        """
        print(f"\n📄 Đang xử lý: {csv_file.name}")

        processed_chunks = []
        chunk_iterator = pd.read_csv(csv_file, chunksize=self.chunk_size,
                                     low_memory=False, encoding='utf-8')

        # Progress bar cho chunks
        if TQDM_AVAILABLE:
            file_size = csv_file.stat().st_size
            estimated_chunks = max(1, file_size // (self.chunk_size * 500))
            chunk_iterator = tqdm(chunk_iterator, desc="   Chunks",
                                  total=estimated_chunks, unit="chunk")

        for chunk in chunk_iterator:
            self.stats['total_rows_read'] += len(chunk)

            # Bước 1: Chuẩn hóa tên cột
            chunk = self._clean_column_names(chunk)

            # Bước 2: Loại bỏ cột identification
            chunk = self._drop_identification_columns(chunk)

            # Bước 3: Chuyển đổi sang dạng số
            chunk = self._convert_to_numeric(chunk)

            # Bước 4: Chuyển đổi nhãn sang binary
            chunk = self._convert_to_binary_label(chunk)

            # Bước 5: Loại bỏ zero-variance columns
            chunk = self._drop_zero_variance_columns(chunk)

            # Bước 6: Xử lý NaN và Inf bằng Mode
            chunk = self._handle_nan_inf_with_mode(chunk)

            processed_chunks.append(chunk)
            gc.collect()

        # Gộp các chunks lại
        if processed_chunks:
            df = pd.concat(processed_chunks, ignore_index=True)
            del processed_chunks
            gc.collect()
            return df

        return None

    def clean_all_files(self):
        """
        Clean tất cả các file CSV

        Returns:
            DataFrame đã clean hoàn chỉnh
        """
        start_time = datetime.now()
        print("\n" + "="*80)
        print(" BẮT ĐẦU CLEAN DỮ LIỆU CICIDS2018")
        print("="*80)

        csv_files = self._get_csv_files()

        # Bước 1: Thu thập thông tin (mode, zero-variance)
        all_columns = self._first_pass_collect_info(csv_files)

        # Bước 2: Xử lý từng file
        print("\n" + "="*80)
        print(" BƯỚC 2: CLEAN DỮ LIỆU")
        print("="*80)

        all_dataframes = []
        for csv_file in csv_files:
            df = self._process_single_file(csv_file)
            if df is not None:
                all_dataframes.append(df)
                print(f"   ✅ Đã xử lý: {len(df):,} mẫu")

        # Gộp tất cả lại
        print("\n" + "-"*80)
        print(" ĐANG GỘP DỮ LIỆU...")

        df_combined = pd.concat(all_dataframes, ignore_index=True)
        del all_dataframes
        gc.collect()

        print(f"   Tổng số mẫu sau khi gộp: {len(df_combined):,}")

        # Loại bỏ duplicate trên toàn bộ dataset
        print("   Đang loại bỏ duplicate...")
        rows_before = len(df_combined)
        df_combined = df_combined.drop_duplicates()
        rows_after = len(df_combined)
        self.stats['duplicates_removed'] = rows_before - rows_after
        print(f"   Số mẫu sau khi loại duplicate: {len(df_combined):,}")
        print(f"   Số duplicate đã loại: {self.stats['duplicates_removed']:,}")

        # Đếm số lượng mỗi class
        self.stats['benign_count'] = int((df_combined['binary_label'] == 0).sum())
        self.stats['attack_count'] = int((df_combined['binary_label'] == 1).sum())

        # Cập nhật thống kê
        self.stats['rows_after_cleaning'] = len(df_combined)
        self.stats['feature_count'] = len(df_combined.columns) - 1  # Trừ cột label

        # Lưu tên features
        self.feature_names = [col for col in df_combined.columns if col != 'binary_label']

        end_time = datetime.now()
        self.stats['processing_time'] = (end_time - start_time).total_seconds()

        return df_combined

    def save_cleaned_data(self, df):
        """
        Lưu dữ liệu đã clean

        Lưu thành các file:
        - cleaned_data.parquet (dữ liệu đã clean, chưa normalize)
        - feature_names.txt
        - cleaning_metadata.json
        """
        print("\n" + "="*80)
        print(" ĐANG LƯU DỮ LIỆU ĐÃ CLEAN...")
        print("="*80)

        # Lưu dữ liệu dạng parquet (nhanh và nhỏ gọn)
        parquet_path = self.output_dir / 'cleaned_data.parquet'
        df.to_parquet(parquet_path, index=False)
        print(f"   ✅ Đã lưu: {parquet_path}")
        print(f"      Kích thước file: {parquet_path.stat().st_size / (1024*1024):.2f} MB")

        # Cũng lưu dạng CSV để dễ kiểm tra (optional, có thể comment nếu file quá lớn)
        # csv_path = self.output_dir / 'cleaned_data.csv'
        # df.to_csv(csv_path, index=False)
        # print(f"   ✅ Đã lưu: {csv_path}")

        # Lưu feature names
        with open(self.output_dir / 'feature_names.txt', 'w') as f:
            for name in self.feature_names:
                f.write(name + '\n')
        print(f"   ✅ Đã lưu: feature_names.txt")

        # Lưu column modes (để có thể sử dụng cho dữ liệu mới)
        with open(self.output_dir / 'column_modes.pkl', 'wb') as f:
            pickle.dump(self.column_modes, f)
        print(f"   ✅ Đã lưu: column_modes.pkl")

        # Lưu zero-variance columns
        with open(self.output_dir / 'zero_variance_cols.pkl', 'wb') as f:
            pickle.dump(self.zero_variance_cols, f)
        print(f"   ✅ Đã lưu: zero_variance_cols.pkl")

        # Chuyển đổi stats sang kiểu Python native
        stats_native = {}
        for key, value in self.stats.items():
            if hasattr(value, 'item'):
                stats_native[key] = value.item()
            elif isinstance(value, (np.integer, np.floating)):
                stats_native[key] = int(value) if isinstance(value, np.integer) else float(value)
            else:
                stats_native[key] = value

        # Lưu metadata
        metadata = {
            'n_features': len(self.feature_names),
            'feature_names': self.feature_names,
            'total_samples': int(len(df)),
            'benign_count': self.stats['benign_count'],
            'attack_count': self.stats['attack_count'],
            'benign_ratio': self.stats['benign_count'] / len(df),
            'attack_ratio': self.stats['attack_count'] / len(df),
            'zero_variance_cols': self.zero_variance_cols,
            'stats': stats_native,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        with open(self.output_dir / 'cleaning_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=4, ensure_ascii=False)
        print(f"   ✅ Đã lưu: cleaning_metadata.json")

        print(f"\n📁 Tất cả file được lưu tại: {self.output_dir}")

    def print_summary(self):
        """In tóm tắt quá trình xử lý"""
        print("\n" + "="*80)
        print(" TÓM TẮT CLEAN DỮ LIỆU")
        print("="*80)
        print(f"   Tổng số dòng đọc được:        {self.stats['total_rows_read']:,}")
        print(f"   Số dòng sau khi clean:        {self.stats['rows_after_cleaning']:,}")
        print(f"   Số duplicate đã loại:         {self.stats['duplicates_removed']:,}")
        print(f"   Số NaN đã thay bằng mode:     {self.stats['nan_replaced']:,}")
        print(f"   Số Inf đã thay bằng mode:     {self.stats['inf_replaced']:,}")
        print(f"   Số cột zero-variance đã loại: {self.stats['zero_variance_cols_removed']}")
        print(f"   Số features còn lại:          {self.stats['feature_count']}")
        print(f"\n   📈 PHÂN BỐ NHÃN:")
        print(f"   Số mẫu Benign (0):  {self.stats['benign_count']:,} ({self.stats['benign_count']/self.stats['rows_after_cleaning']*100:.1f}%)")
        print(f"   Số mẫu Attack (1):  {self.stats['attack_count']:,} ({self.stats['attack_count']/self.stats['rows_after_cleaning']*100:.1f}%)")
        print(f"\n   Thời gian xử lý: {self.stats['processing_time']:.2f} giây")
        print("="*80)


def main():
    """Hàm chính để chạy cleaning"""

    print("\n" + "="*80)
    print("🧹 BƯỚC 1: CLEAN DỮ LIỆU CICIDS2018 CHO CNN")
    print("   Phát hiện lưu lượng mạng IoT bất thường")
    print("="*80)

    # Khởi tạo cleaner
    cleaner = CICIDS2018_DataCleaner(
        data_dir=DATA_DIR,
        output_dir=OUTPUT_DIR,
        chunk_size=CHUNK_SIZE
    )

    # Clean tất cả các file
    df = cleaner.clean_all_files()

    # Lưu dữ liệu đã clean
    cleaner.save_cleaned_data(df)

    # In tóm tắt
    cleaner.print_summary()

    print("\n✅ HOÀN THÀNH BƯỚC 1!")
    print("   Dữ liệu đã được clean và lưu vào folder.")
    print("   Chạy step2_prepare_training_data.py để chia train/val/test và cân bằng dữ liệu.")

    return cleaner


if __name__ == "__main__":
    cleaner = main()



🌐 Đang chạy trên KAGGLE

🧹 BƯỚC 1: CLEAN DỮ LIỆU CICIDS2018 CHO CNN
   Phát hiện lưu lượng mạng IoT bất thường

 BẮT ĐẦU CLEAN DỮ LIỆU CICIDS2018

📂 Tìm thấy 10 file CSV:
   - Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
   - Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
   - Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
   - Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
   - Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
   - Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
   - Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
   - Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
   - Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
   - Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv

📊 BƯỚC 1: THU THẬP THÔNG TIN TỪ DỮ LIỆU

   Đang scan: Friday-02-03-2018_TrafficForML_CICFlowMeter.csv

   Đang scan: Friday-16-02-2018_TrafficForML_CICFlowMeter.csv

   Đang scan: Friday-23-02-2018_TrafficForML_CICFlowMeter.csv

   Đang scan: Thuesday-20-02-2018_TrafficForML_CICF

   Chunks: 4chunk [00:13,  3.30s/chunk]                    


   ✅ Đã xử lý: 1,048,575 mẫu

📄 Đang xử lý: Friday-16-02-2018_TrafficForML_CICFlowMeter.csv


   Chunks: 4chunk [00:17,  4.43s/chunk]                    


   ✅ Đã xử lý: 1,048,574 mẫu

📄 Đang xử lý: Friday-23-02-2018_TrafficForML_CICFlowMeter.csv


...

   ✅ Đã xử lý: 1,048,575 mẫu

📄 Đang xử lý: Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv


   Chunks: 4chunk [00:10,  2.65s/chunk]                    


   ✅ Đã xử lý: 1,048,575 mẫu

📄 Đang xử lý: Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


   Chunks: 3chunk [00:29,  9.83s/chunk]                    


   ✅ Đã xử lý: 613,071 mẫu

--------------------------------------------------------------------------------
 ĐANG GỘP DỮ LIỆU...
   Tổng số mẫu sau khi gộp: 16,232,943
   Đang loại bỏ duplicate...
   Số mẫu sau khi loại duplicate: 10,822,059
   Số duplicate đã loại: 5,410,884

 ĐANG LƯU DỮ LIỆU ĐÃ CLEAN...
   ✅ Đã lưu: /kaggle/working/cleaned_data/cleaned_data.parquet
      Kích thước file: 1479.20 MB
   ✅ Đã lưu: feature_names.txt
   ✅ Đã lưu: column_modes.pkl
   ✅ Đã lưu: zero_variance_cols.pkl
   ✅ Đã lưu: cleaning_metadata.json

📁 Tất cả file được lưu tại: /kaggle/working/cleaned_data

 TÓM TẮT CLEAN DỮ LIỆU
   Tổng số dòng đọc được:        16,233,002
   Số dòng sau khi clean:        10,822,059
   Số duplicate đã loại:         5,410,884
   Số NaN đã thay bằng mode:     59,721
   Số Inf đã thay bằng mode:     131,799
   Số cột zero-variance đã loại: 8
   Số features còn lại:          69

   📈 PHÂN BỐ NHÃN:
   Số mẫu Benign (0):  9,496,113 (87.7%)
   Số mẫu Attack (1):  1,325,946 (1

In [2]:
"""
======================================================================================
BƯỚC 2: CHUẨN BỊ DỮ LIỆU TRAINING CHO CNN - CÂN BẰNG VÀ CHIA TRAIN/VAL/TEST
======================================================================================

Script này thực hiện:
1. Đọc dữ liệu đã clean từ step1
2. Cân bằng số lượng nhãn (70% Benign, 30% Attack hoặc tỷ lệ tùy chỉnh)
3. Áp dụng Log Transform: log_e(1+x)
4. Chuẩn hóa bằng StandardScaler
5. Reshape cho CNN 1D
6. Chia train/val/test với stratify để giữ tỷ lệ
7. Lưu dữ liệu đã xử lý để train

Có thể chạy trên cả Kaggle và Local
"""

import os
import numpy as np
import pandas as pd
import pickle
import json
import gc
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# THƯ VIỆN CHUẨN HÓA VÀ XỬ LÝ DỮ LIỆU
# ============================================================================
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Kiểm tra môi trường chạy (Kaggle hoặc Local)
IS_KAGGLE = os.path.exists('/kaggle/input')

# ============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================================
if IS_KAGGLE:
    CLEANED_DATA_DIR = "/kaggle/working/cleaned_data"
    OUTPUT_DIR = "/kaggle/working/training_data"
    print("🌐 Đang chạy trên KAGGLE")
else:
    CLEANED_DATA_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\cleaned_data"
    OUTPUT_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\training_data"
    print("💻 Đang chạy trên LOCAL")

# ============================================================================
# CẤU HÌNH CÂN BẰNG DỮ LIỆU
# ============================================================================

# Random state để tái tạo kết quả
RANDOM_STATE = 42

# Tổng số mẫu mong muốn (train + val + test)
TOTAL_SAMPLES = 3000000  # 3 triệu mẫu

# Tỷ lệ phần trăm cho mỗi class
BENIGN_RATIO = 0.70  # 70% Benign
ATTACK_RATIO = 0.30  # 30% Attack

# Tính số lượng mẫu cho mỗi class
TARGET_BENIGN = int(TOTAL_SAMPLES * BENIGN_RATIO)  # 2,100,000
TARGET_ATTACK = int(TOTAL_SAMPLES * ATTACK_RATIO)  # 900,000

# Tỷ lệ chia train/val/test
TEST_SIZE = 0.20   # 20% cho test
VAL_SIZE = 0.10    # 10% cho validation (từ tổng)
# Train sẽ là 70%

# ============================================================================
# CLASS CHUẨN BỊ DỮ LIỆU TRAINING
# ============================================================================

class TrainingDataPreparer:
    """
    Class chuẩn bị dữ liệu training cho CNN

    Các bước:
    1. Đọc dữ liệu đã clean
    2. Cân bằng dữ liệu theo tỷ lệ mong muốn
    3. Áp dụng log transform: log_e(1+x)
    4. Chuẩn hóa bằng StandardScaler
    5. Reshape cho CNN
    6. Chia train/val/test
    7. Lưu dữ liệu
    """

    def __init__(self, cleaned_data_dir, output_dir,
                 total_samples=TOTAL_SAMPLES,
                 benign_ratio=BENIGN_RATIO,
                 attack_ratio=ATTACK_RATIO,
                 test_size=TEST_SIZE,
                 val_size=VAL_SIZE):
        """
        Khởi tạo preparer

        Args:
            cleaned_data_dir: Đường dẫn thư mục chứa dữ liệu đã clean
            output_dir: Đường dẫn thư mục lưu kết quả
            total_samples: Tổng số mẫu mong muốn
            benign_ratio: Tỷ lệ Benign (0-1)
            attack_ratio: Tỷ lệ Attack (0-1)
            test_size: Tỷ lệ test set
            val_size: Tỷ lệ validation set
        """
        self.cleaned_data_dir = Path(cleaned_data_dir)
        self.output_dir = Path(output_dir)
        self.total_samples = total_samples
        self.benign_ratio = benign_ratio
        self.attack_ratio = attack_ratio
        self.test_size = test_size
        self.val_size = val_size

        # Tính target cho mỗi class
        self.target_benign = int(total_samples * benign_ratio)
        self.target_attack = int(total_samples * attack_ratio)

        # Khởi tạo scaler
        self.scaler = StandardScaler()

        # Tạo thư mục output
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # Lưu tên features
        self.feature_names = None

        # Thống kê
        self.stats = {
            'original_benign': 0,
            'original_attack': 0,
            'sampled_benign': 0,
            'sampled_attack': 0,
            'train_samples': 0,
            'val_samples': 0,
            'test_samples': 0,
            'n_features': 0
        }

    def load_cleaned_data(self):
        """Đọc dữ liệu đã clean từ step1"""
        print("\n" + "="*80)
        print("📂 ĐANG ĐỌC DỮ LIỆU ĐÃ CLEAN...")
        print("="*80)

        parquet_path = self.cleaned_data_dir / 'cleaned_data.parquet'

        if not parquet_path.exists():
            raise FileNotFoundError(
                f"Không tìm thấy file {parquet_path}\n"
                f"Hãy chạy step1_clean_data.py trước!"
            )

        df = pd.read_parquet(parquet_path)

        # Đọc feature names
        feature_names_path = self.cleaned_data_dir / 'feature_names.txt'
        if feature_names_path.exists():
            with open(feature_names_path, 'r') as f:
                self.feature_names = [line.strip() for line in f.readlines()]
        else:
            self.feature_names = [col for col in df.columns if col != 'binary_label']

        # Thống kê
        self.stats['original_benign'] = int((df['binary_label'] == 0).sum())
        self.stats['original_attack'] = int((df['binary_label'] == 1).sum())
        self.stats['n_features'] = len(self.feature_names)

        print(f"   ✅ Đã đọc: {len(df):,} mẫu")
        print(f"   📊 Phân bố gốc:")
        print(f"      - Benign: {self.stats['original_benign']:,} ({self.stats['original_benign']/len(df)*100:.1f}%)")
        print(f"      - Attack: {self.stats['original_attack']:,} ({self.stats['original_attack']/len(df)*100:.1f}%)")
        print(f"   📋 Số features: {self.stats['n_features']}")

        return df

    def balanced_sample(self, df):
        """
        Sample dữ liệu với tỷ lệ cân bằng mong muốn

        Chiến lược:
        - Nếu có đủ mẫu: lấy đúng số lượng target
        - Nếu không đủ Attack: giảm Benign tương ứng để giữ tỷ lệ
        - Nếu không đủ cả hai: lấy tối đa có thể với tỷ lệ đúng
        """
        print("\n" + "="*80)
        print("⚖️ ĐANG CÂN BẰNG DỮ LIỆU...")
        print("="*80)

        # Tách theo class
        df_benign = df[df['binary_label'] == 0]
        df_attack = df[df['binary_label'] == 1]

        n_benign = len(df_benign)
        n_attack = len(df_attack)

        print(f"\n   🎯 Target mong muốn:")
        print(f"      - Tổng: {self.total_samples:,}")
        print(f"      - Benign: {self.target_benign:,} ({self.benign_ratio*100:.0f}%)")
        print(f"      - Attack: {self.target_attack:,} ({self.attack_ratio*100:.0f}%)")

        # Xác định số lượng thực tế có thể lấy
        # Ưu tiên giữ đúng tỷ lệ
        actual_attack = min(self.target_attack, n_attack)
        # Tính Benign dựa trên Attack thực tế để giữ tỷ lệ
        actual_benign = int(actual_attack * (self.benign_ratio / self.attack_ratio))
        actual_benign = min(actual_benign, n_benign)

        # Nếu Benign bị giới hạn, điều chỉnh Attack
        if actual_benign < int(actual_attack * (self.benign_ratio / self.attack_ratio)):
            actual_attack = int(actual_benign * (self.attack_ratio / self.benign_ratio))

        print(f"\n   📊 Số lượng thực tế sẽ lấy:")
        print(f"      - Benign: {actual_benign:,}")
        print(f"      - Attack: {actual_attack:,}")
        print(f"      - Tổng: {actual_benign + actual_attack:,}")
        print(f"      - Tỷ lệ thực tế: {actual_benign/(actual_benign+actual_attack)*100:.1f}% - {actual_attack/(actual_benign+actual_attack)*100:.1f}%")

        if actual_benign < self.target_benign or actual_attack < self.target_attack:
            print(f"\n   ⚠️ Không đủ mẫu để đạt target!")
            print(f"      Có sẵn: Benign={n_benign:,}, Attack={n_attack:,}")

        # Random sample từ mỗi class
        print(f"\n   🔄 Đang sample...")

        # Sử dụng random sampling
        df_benign_sampled = df_benign.sample(n=actual_benign, random_state=RANDOM_STATE)
        df_attack_sampled = df_attack.sample(n=actual_attack, random_state=RANDOM_STATE)

        # Gộp lại và shuffle
        df_balanced = pd.concat([df_benign_sampled, df_attack_sampled], ignore_index=True)
        df_balanced = df_balanced.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

        # Cập nhật stats
        self.stats['sampled_benign'] = actual_benign
        self.stats['sampled_attack'] = actual_attack

        print(f"\n   ✅ Kết quả sau khi cân bằng:")
        print(f"      - Benign: {actual_benign:,} ({actual_benign/(actual_benign+actual_attack)*100:.1f}%)")
        print(f"      - Attack: {actual_attack:,} ({actual_attack/(actual_benign+actual_attack)*100:.1f}%)")
        print(f"      - Tổng: {len(df_balanced):,}")

        # Giải phóng bộ nhớ
        del df_benign, df_attack, df_benign_sampled, df_attack_sampled
        gc.collect()

        return df_balanced

    def apply_log_transform(self, X):
        """
        Áp dụng Log Transform: log_e(1+x)

        Lưu ý: log(1+x) giúp:
        - Giảm skewness của dữ liệu
        - Xử lý các giá trị lớn
        - Bảo toàn giá trị 0 (log(1+0) = 0)
        """
        print("\n🔢 ĐANG ÁP DỤNG LOG TRANSFORM: log_e(1+x)...")

        # Đảm bảo không có giá trị âm (log không xác định cho số âm)
        # Với dữ liệu network flow, các giá trị thường >= 0
        # Nếu có giá trị âm, ta shift để min = 0
        min_val = X.min()
        if min_val < 0:
            print(f"   ⚠️ Phát hiện giá trị âm (min={min_val:.4f}), đang shift...")
            X = X - min_val  # Shift để min = 0

        # Áp dụng log(1+x)
        X_log = np.log1p(X)  # log1p(x) = log(1+x), ổn định hơn với x nhỏ

        print(f"   ✅ Log transform hoàn tất")
        print(f"      Range trước: [{X.min():.4f}, {X.max():.4f}]")
        print(f"      Range sau:   [{X_log.min():.4f}, {X_log.max():.4f}]")

        return X_log

    def normalize_features(self, X):
        """
        Chuẩn hóa features bằng StandardScaler
        """
        print("\n📐 ĐANG CHUẨN HÓA BẰNG STANDARDSCALER...")

        X_normalized = self.scaler.fit_transform(X)

        print(f"   ✅ StandardScaler hoàn tất")
        print(f"      Mean: {X_normalized.mean():.6f}")
        print(f"      Std:  {X_normalized.std():.6f}")

        return X_normalized

    def reshape_for_cnn(self, X):
        """
        Reshape dữ liệu cho CNN 1D
        CNN 1D yêu cầu input shape: (samples, features, channels)
        """
        print("\n🔄 ĐANG RESHAPE CHO CNN 1D...")

        X_reshaped = X.reshape(X.shape[0], X.shape[1], 1)

        print(f"   ✅ Shape: {X.shape} -> {X_reshaped.shape}")
        print(f"      (samples, features, channels)")

        return X_reshaped

    def split_data(self, X, y):
        """
        Chia dữ liệu thành train/val/test

        Thêm validation: Train 70%, Val 10%, Test 20%

        Sử dụng stratify để giữ tỷ lệ class trong tất cả các tập
        """
        print("\n📊 ĐANG CHIA DỮ LIỆU TRAIN/VAL/TEST...")

        # Bước 1: Chia train+val / test (80/20)
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y,
            test_size=self.test_size,
            random_state=RANDOM_STATE,
            stratify=y  # Giữ tỷ lệ class
        )

        # Bước 2: Chia train / val
        val_ratio_from_temp = self.val_size / (1 - self.test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp,
            test_size=val_ratio_from_temp,
            random_state=RANDOM_STATE,
            stratify=y_temp
        )

        # Cập nhật stats
        self.stats['train_samples'] = len(X_train)
        self.stats['val_samples'] = len(X_val)
        self.stats['test_samples'] = len(X_test)

        print(f"\n   📈 KẾT QUẢ CHIA DỮ LIỆU:")
        print(f"   {'='*50}")
        print(f"   {'Set':<10} {'Samples':>12} {'Benign':>12} {'Attack':>12}")
        print(f"   {'-'*50}")
        print(f"   {'Train':<10} {len(X_train):>12,} {(y_train==0).sum():>12,} {(y_train==1).sum():>12,}")
        print(f"   {'Val':<10} {len(X_val):>12,} {(y_val==0).sum():>12,} {(y_val==1).sum():>12,}")
        print(f"   {'Test':<10} {len(X_test):>12,} {(y_test==0).sum():>12,} {(y_test==1).sum():>12,}")
        print(f"   {'-'*50}")
        print(f"   {'Total':<10} {len(X_train)+len(X_val)+len(X_test):>12,}")

        # Kiểm tra tỷ lệ
        print(f"\n   📊 TỶ LỆ ATTACK TRONG MỖI TẬP:")
        print(f"      Train: {(y_train==1).sum()/len(y_train)*100:.1f}%")
        print(f"      Val:   {(y_val==1).sum()/len(y_val)*100:.1f}%")
        print(f"      Test:  {(y_test==1).sum()/len(y_test)*100:.1f}%")

        return X_train, X_val, X_test, y_train, y_val, y_test

    def save_training_data(self, X_train, X_val, X_test, y_train, y_val, y_test):
        """
        Lưu dữ liệu training

        Lưu các file:
        - X_train.npy, X_val.npy, X_test.npy
        - y_train.npy, y_val.npy, y_test.npy
        - scaler.pkl
        - training_metadata.json
        - feature_names.txt
        """
        print("\n" + "="*80)
        print("💾 ĐANG LƯU DỮ LIỆU TRAINING...")
        print("="*80)

        # Lưu numpy arrays
        np.save(self.output_dir / 'X_train.npy', X_train)
        np.save(self.output_dir / 'X_val.npy', X_val)
        np.save(self.output_dir / 'X_test.npy', X_test)
        np.save(self.output_dir / 'y_train.npy', y_train)
        np.save(self.output_dir / 'y_val.npy', y_val)
        np.save(self.output_dir / 'y_test.npy', y_test)

        print(f"   ✅ X_train.npy: {X_train.shape}")
        print(f"   ✅ X_val.npy:   {X_val.shape}")
        print(f"   ✅ X_test.npy:  {X_test.shape}")
        print(f"   ✅ y_train.npy: {y_train.shape}")
        print(f"   ✅ y_val.npy:   {y_val.shape}")
        print(f"   ✅ y_test.npy:  {y_test.shape}")

        # Lưu scaler
        with open(self.output_dir / 'scaler.pkl', 'wb') as f:
            pickle.dump(self.scaler, f)
        print(f"   ✅ scaler.pkl")

        # Lưu feature names
        with open(self.output_dir / 'feature_names.txt', 'w') as f:
            for name in self.feature_names:
                f.write(name + '\n')
        print(f"   ✅ feature_names.txt")

        # Chuẩn bị metadata
        metadata = {
            'n_features': len(self.feature_names),
            'input_shape': [int(X_train.shape[1]), int(X_train.shape[2])],
            'train_samples': int(X_train.shape[0]),
            'val_samples': int(X_val.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'total_samples': int(X_train.shape[0] + X_val.shape[0] + X_test.shape[0]),
            'class_distribution': {
                'train': {
                    'benign': int((y_train == 0).sum()),
                    'attack': int((y_train == 1).sum())
                },
                'val': {
                    'benign': int((y_val == 0).sum()),
                    'attack': int((y_val == 1).sum())
                },
                'test': {
                    'benign': int((y_test == 0).sum()),
                    'attack': int((y_test == 1).sum())
                }
            },
            'benign_ratio': float(self.benign_ratio),
            'attack_ratio': float(self.attack_ratio),
            'preprocessing': {
                'log_transform': 'log_e(1+x)',
                'normalization': 'StandardScaler'
            },
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        with open(self.output_dir / 'training_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=4, ensure_ascii=False)
        print(f"   ✅ training_metadata.json")

        print(f"\n📁 Tất cả file được lưu tại: {self.output_dir}")

    def calculate_class_weights(self, y_train):
        """
        Tính class weights cho training

        Sử dụng khi dữ liệu vẫn còn imbalanced
        """
        from sklearn.utils.class_weight import compute_class_weight

        classes = np.unique(y_train)
        weights = compute_class_weight('balanced', classes=classes, y=y_train)
        class_weights = dict(zip(classes, weights))

        print(f"\n⚖️ CLASS WEIGHTS (cho training):")
        print(f"   Class 0 (Benign): {class_weights[0]:.4f}")
        print(f"   Class 1 (Attack): {class_weights[1]:.4f}")

        # Lưu class weights
        with open(self.output_dir / 'class_weights.pkl', 'wb') as f:
            pickle.dump(class_weights, f)
        print(f"   ✅ Đã lưu class_weights.pkl")

        return class_weights


def main():
    """Hàm chính"""

    print("\n" + "="*80)
    print("📊 BƯỚC 2: CHUẨN BỊ DỮ LIỆU TRAINING CHO CNN")
    print("   Cân bằng và chia train/val/test")
    print("="*80)

    print(f"\n📋 CẤU HÌNH:")
    print(f"   - Tổng mẫu mong muốn: {TOTAL_SAMPLES:,}")
    print(f"   - Tỷ lệ Benign: {BENIGN_RATIO*100:.0f}%")
    print(f"   - Tỷ lệ Attack: {ATTACK_RATIO*100:.0f}%")
    print(f"   - Train/Val/Test: {(1-TEST_SIZE-VAL_SIZE)*100:.0f}%/{VAL_SIZE*100:.0f}%/{TEST_SIZE*100:.0f}%")

    # Khởi tạo preparer
    preparer = TrainingDataPreparer(
        cleaned_data_dir=CLEANED_DATA_DIR,
        output_dir=OUTPUT_DIR,
        total_samples=TOTAL_SAMPLES,
        benign_ratio=BENIGN_RATIO,
        attack_ratio=ATTACK_RATIO,
        test_size=TEST_SIZE,
        val_size=VAL_SIZE
    )

    # Bước 1: Đọc dữ liệu đã clean
    df = preparer.load_cleaned_data()

    # Bước 2: Cân bằng dữ liệu
    df = preparer.balanced_sample(df)

    # Tách features và labels
    X = df.drop(columns=['binary_label']).values
    y = df['binary_label'].values

    # Giải phóng bộ nhớ DataFrame
    del df
    gc.collect()

    # Bước 3: Áp dụng Log Transform
    X = preparer.apply_log_transform(X)

    # Bước 4: Chuẩn hóa
    X = preparer.normalize_features(X)

    # Bước 5: Reshape cho CNN
    X = preparer.reshape_for_cnn(X)

    # Bước 6: Chia train/val/test
    X_train, X_val, X_test, y_train, y_val, y_test = preparer.split_data(X, y)

    # Giải phóng bộ nhớ
    del X, y
    gc.collect()

    # Bước 7: Tính class weights
    class_weights = preparer.calculate_class_weights(y_train)

    # Bước 8: Lưu dữ liệu
    preparer.save_training_data(X_train, X_val, X_test, y_train, y_val, y_test)

    print("\n" + "="*80)
    print("✅ HOÀN THÀNH BƯỚC 2!")
    print("   Dữ liệu đã sẵn sàng cho việc huấn luyện CNN.")
    print("   Chạy step3_train_cnn.py để train mô hình.")
    print("="*80)

    return preparer


if __name__ == "__main__":
    preparer = main()



🌐 Đang chạy trên KAGGLE

📊 BƯỚC 2: CHUẨN BỊ DỮ LIỆU TRAINING CHO CNN
   Cân bằng và chia train/val/test

📋 CẤU HÌNH:
   - Tổng mẫu mong muốn: 3,000,000
   - Tỷ lệ Benign: 70%
   - Tỷ lệ Attack: 30%
   - Train/Val/Test: 70%/10%/20%

📂 ĐANG ĐỌC DỮ LIỆU ĐÃ CLEAN...
   ✅ Đã đọc: 10,822,059 mẫu
   📊 Phân bố gốc:
      - Benign: 9,496,113 (87.7%)
      - Attack: 1,325,946 (12.3%)
   📋 Số features: 69

⚖️ ĐANG CÂN BẰNG DỮ LIỆU...

   🎯 Target mong muốn:
      - Tổng: 3,000,000
      - Benign: 2,100,000 (70%)
      - Attack: 900,000 (30%)

   📊 Số lượng thực tế sẽ lấy:
      - Benign: 2,100,000
      - Attack: 900,000
      - Tổng: 3,000,000
      - Tỷ lệ thực tế: 70.0% - 30.0%

   🔄 Đang sample...

   ✅ Kết quả sau khi cân bằng:
      - Benign: 2,100,000 (70.0%)
      - Attack: 900,000 (30.0%)
      - Tổng: 3,000,000

🔢 ĐANG ÁP DỤNG LOG TRANSFORM: log_e(1+x)...
   ⚠️ Phát hiện giá trị âm (min=-947405000000.0000), đang shift...
   ✅ Log transform hoàn tất
      Range trước: [0.0000, 1927186000

In [6]:
"""
======================================================================================
BƯỚC 3: TRAIN MÔ HÌNH CNN CHO PHÁT HIỆN LƯU LƯỢNG MẠNG IOT BẤT THƯỜNG
======================================================================================

Kiến trúc CNN theo yêu cầu:
- Input Layer: Shape (num_features, 1)
- Conv1D (32 filters, kernel 2) -> MaxPooling1D (2)
- Conv1D (32 filters, kernel 2) -> MaxPooling1D (2)
- Conv1D (64 filters, kernel 2) -> MaxPooling1D (2)
- Conv1D (64 filters, kernel 2) -> MaxPooling1D (2)
- Conv1D (64 filters, kernel 2) -> MaxPooling1D (2)
- BatchNormalization + Dropout (0.5)
- Flatten
- Dense(1, activation='sigmoid')

Loss: binary_crossentropy
Optimizer: Adam
Metrics: Accuracy, Precision, Recall

Có thể chạy trên cả Kaggle và Local
"""

import os
import numpy as np
import pickle
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# TENSORFLOW/KERAS
# ============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, Flatten, Dense,
    Dropout, BatchNormalization, Input
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
)
from tensorflow.keras.metrics import Precision, Recall

# Kiểm tra GPU
print("="*80)
print("🖥️ THÔNG TIN HỆ THỐNG")
print("="*80)
print(f"TensorFlow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU available: {len(gpus)} GPU(s)")
    for gpu in gpus:
        print(f"   - {gpu}")
    # Cấu hình GPU memory growth để tránh chiếm hết bộ nhớ
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("⚠️ Không có GPU, sẽ sử dụng CPU")

# Kiểm tra môi trường chạy
IS_KAGGLE = os.path.exists('/kaggle/input')

# ============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================================
if IS_KAGGLE:
    TRAINING_DATA_DIR = "/kaggle/working/training_data"
    MODEL_DIR = "/kaggle/working/models"
    LOG_DIR = "/kaggle/working/logs"
    print("🌐 Đang chạy trên KAGGLE")
else:
    TRAINING_DATA_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\training_data"
    MODEL_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\models"
    LOG_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\logs"
    print("💻 Đang chạy trên LOCAL")

# ============================================================================
# CẤU HÌNH HUẤN LUYỆN
# ============================================================================

# Hyperparameters
BATCH_SIZE = 256        # Batch size cho training
EPOCHS = 50             # Số epochs tối đa
LEARNING_RATE = 0.001   # Learning rate ban đầu

# Regularization
DROPOUT_RATE = 0.5      # Dropout rate trước Flatten

# Early stopping
PATIENCE = 10           # Số epochs chờ trước khi dừng

# Random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


# ============================================================================
# HÀM XÂY DỰNG MÔ HÌNH CNN
# ============================================================================

def build_cnn_model(input_shape):
    """
    Xây dựng mô hình CNN cho phân loại binary

    Kiến trúc theo yêu cầu:
    - 5 lớp Conv1D với MaxPooling
    - BatchNormalization và Dropout trước Flatten
    - Output layer với sigmoid activation

    Args:
        input_shape: Shape của input (n_features, 1)

    Returns:
        model: Keras Sequential model
    """
    print("\n" + "="*80)
    print("🏗️ ĐANG XÂY DỰNG MÔ HÌNH CNN")
    print("="*80)
    print(f"   Input shape: {input_shape}")

    model = Sequential(name='CNN_Binary_Classification')

    # Input layer
    model.add(Input(shape=input_shape))

    # ========== KHỐI CONV 1 ==========
    # Conv1D (32 filters, kernel 2x1) -> MaxPooling1D (2)
    model.add(Conv1D(
        filters=32,
        kernel_size=2,
        activation='relu',
        padding='same',  # Giữ nguyên kích thước
        name='conv1d_1'
    ))
    model.add(MaxPooling1D(pool_size=2, name='maxpool_1'))

    # ========== KHỐI CONV 2 ==========
    # Conv1D (32 filters, kernel 2x1) -> MaxPooling1D (2)
    model.add(Conv1D(
        filters=32,
        kernel_size=2,
        activation='relu',
        padding='same',
        name='conv1d_2'
    ))
    model.add(MaxPooling1D(pool_size=2, name='maxpool_2'))

    # ========== KHỐI CONV 3 ==========
    # Conv1D (64 filters, kernel 2x1) -> MaxPooling1D (2)
    model.add(Conv1D(
        filters=64,
        kernel_size=2,
        activation='relu',
        padding='same',
        name='conv1d_3'
    ))
    model.add(MaxPooling1D(pool_size=2, name='maxpool_3'))

    # ========== KHỐI CONV 4 ==========
    # Conv1D (64 filters, kernel 2x1) -> MaxPooling1D (2)
    model.add(Conv1D(
        filters=64,
        kernel_size=2,
        activation='relu',
        padding='same',
        name='conv1d_4'
    ))
    model.add(MaxPooling1D(pool_size=2, name='maxpool_4'))

    # ========== KHỐI CONV 5 ==========
    # Conv1D (64 filters, kernel 2x1) -> MaxPooling1D (2)
    model.add(Conv1D(
        filters=64,
        kernel_size=2,
        activation='relu',
        padding='same',
        name='conv1d_5'
    ))
    model.add(MaxPooling1D(pool_size=2, name='maxpool_5'))

    # ========== REGULARIZATION ==========
    # BatchNormalization và Dropout trước Flatten
    model.add(BatchNormalization(name='batch_norm'))
    model.add(Dropout(DROPOUT_RATE, name='dropout'))

    # ========== FLATTEN ==========
    model.add(Flatten(name='flatten'))

    # ========== OUTPUT LAYER ==========
    # Dense(1, activation='sigmoid') cho binary classification
    model.add(Dense(1, activation='sigmoid', name='output'))

    # ========== COMPILE ==========
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            Precision(name='precision'),
            Recall(name='recall')
        ]
    )

    # In tóm tắt mô hình
    print("\n   📋 KIẾN TRÚC MÔ HÌNH:")
    model.summary()

    return model


def load_training_data(data_dir):
    """
    Load dữ liệu training đã được chuẩn bị từ step 2

    Args:
        data_dir: Đường dẫn thư mục chứa dữ liệu

    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test, class_weights
    """
    print("\n" + "="*80)
    print("📂 ĐANG LOAD DỮ LIỆU TRAINING...")
    print("="*80)

    data_dir = Path(data_dir)

    # Load numpy arrays
    X_train = np.load(data_dir / 'X_train.npy')
    X_val = np.load(data_dir / 'X_val.npy')
    X_test = np.load(data_dir / 'X_test.npy')
    y_train = np.load(data_dir / 'y_train.npy')
    y_val = np.load(data_dir / 'y_val.npy')
    y_test = np.load(data_dir / 'y_test.npy')

    print(f"   ✅ X_train: {X_train.shape}")
    print(f"   ✅ X_val:   {X_val.shape}")
    print(f"   ✅ X_test:  {X_test.shape}")
    print(f"   ✅ y_train: {y_train.shape}")
    print(f"   ✅ y_val:   {y_val.shape}")
    print(f"   ✅ y_test:  {y_test.shape}")

    # Load class weights nếu có
    class_weights = None
    class_weights_path = data_dir / 'class_weights.pkl'
    if class_weights_path.exists():
        with open(class_weights_path, 'rb') as f:
            class_weights = pickle.load(f)
        print(f"\n   ⚖️ Class weights loaded:")
        print(f"      Class 0 (Benign): {class_weights[0]:.4f}")
        print(f"      Class 1 (Attack): {class_weights[1]:.4f}")

    # Thống kê phân bố
    print(f"\n   📊 PHÂN BỐ DỮ LIỆU:")
    for name, y in [('Train', y_train), ('Val', y_val), ('Test', y_test)]:
        benign = (y == 0).sum()
        attack = (y == 1).sum()
        total = len(y)
        print(f"      {name}: Benign={benign:,} ({benign/total*100:.1f}%), Attack={attack:,} ({attack/total*100:.1f}%)")

    return X_train, X_val, X_test, y_train, y_val, y_test, class_weights


def create_callbacks(model_dir, log_dir):
    """
    Tạo các callbacks cho training

    Callbacks:
    - EarlyStopping: Dừng sớm khi val_loss không giảm
    - ModelCheckpoint: Lưu model tốt nhất
    - ReduceLROnPlateau: Giảm learning rate khi plateau
    - TensorBoard: Logging cho visualization
    """
    print("\n📌 ĐANG CẤU HÌNH CALLBACKS...")

    model_dir = Path(model_dir)
    log_dir = Path(log_dir)
    model_dir.mkdir(parents=True, exist_ok=True)
    log_dir.mkdir(parents=True, exist_ok=True)

    callbacks = []

    # 1. Early Stopping
    # Dừng training khi val_loss không cải thiện sau PATIENCE epochs
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=PATIENCE,
        verbose=1,
        mode='min',
        restore_best_weights=True  # Khôi phục weights tốt nhất
    )
    callbacks.append(early_stopping)
    print(f"   ✅ EarlyStopping: patience={PATIENCE}")

    # 2. Model Checkpoint
    # Lưu model có val_loss thấp nhất
    checkpoint_path = model_dir / 'best_model.keras'
    model_checkpoint = ModelCheckpoint(
        filepath=str(checkpoint_path),
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks.append(model_checkpoint)
    print(f"   ✅ ModelCheckpoint: {checkpoint_path}")

    # 3. Reduce Learning Rate on Plateau
    # Giảm LR khi val_loss không giảm
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,        # Giảm LR còn 1/2
        patience=5,        # Chờ 5 epochs
        min_lr=1e-7,       # LR tối thiểu
        verbose=1
    )
    callbacks.append(reduce_lr)
    print(f"   ✅ ReduceLROnPlateau: factor=0.5, patience=5")

    # 4. TensorBoard (optional)
    tensorboard_log = log_dir / datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard = TensorBoard(
        log_dir=str(tensorboard_log),
        histogram_freq=1
    )
    callbacks.append(tensorboard)
    print(f"   ✅ TensorBoard: {tensorboard_log}")

    return callbacks


def train_model(model, X_train, y_train, X_val, y_val, class_weights, callbacks):
    """
    Huấn luyện mô hình

    Args:
        model: Keras model
        X_train, y_train: Dữ liệu training
        X_val, y_val: Dữ liệu validation
        class_weights: Dictionary class weights
        callbacks: List các callbacks

    Returns:
        history: Training history
    """
    print("\n" + "="*80)
    print("🚀 BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH")
    print("="*80)
    print(f"   Epochs: {EPOCHS}")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Learning rate: {LEARNING_RATE}")
    print(f"   Class weights: {'Có' if class_weights else 'Không'}")

    start_time = datetime.now()

    history = model.fit(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        class_weight=class_weights,  # Sử dụng class weights để xử lý imbalance
        callbacks=callbacks,
        verbose=1
    )

    end_time = datetime.now()
    training_time = (end_time - start_time).total_seconds()

    print(f"\n   ⏱️ Thời gian training: {training_time/60:.2f} phút")
    print(f"   📈 Best val_loss: {min(history.history['val_loss']):.4f}")
    print(f"   📈 Best val_accuracy: {max(history.history['val_accuracy']):.4f}")

    return history, training_time


def evaluate_model(model, X_test, y_test):
    """
    Đánh giá mô hình trên test set

    Args:
        model: Trained model
        X_test, y_test: Dữ liệu test

    Returns:
        results: Dictionary kết quả đánh giá
    """
    print("\n" + "="*80)
    print("📊 ĐÁNH GIÁ MÔ HÌNH TRÊN TEST SET")
    print("="*80)

    # Evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=1)

    # Tính F1-score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-7)

    results = {
        'test_loss': float(loss),
        'test_accuracy': float(accuracy),
        'test_precision': float(precision),
        'test_recall': float(recall),
        'test_f1_score': float(f1_score)
    }

    print(f"\n   📊 KẾT QUẢ:")
    print(f"   {'='*40}")
    print(f"   Loss:      {loss:.4f}")
    print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1_score:.4f}")
    print(f"   {'='*40}")

    # Predictions cho confusion matrix
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    # Confusion matrix
    from sklearn.metrics import confusion_matrix, classification_report

    cm = confusion_matrix(y_test, y_pred)
    print(f"\n   📋 CONFUSION MATRIX:")
    print(f"                 Predicted")
    print(f"                 Benign  Attack")
    print(f"   Actual Benign  {cm[0,0]:>6}  {cm[0,1]:>6}")
    print(f"   Actual Attack  {cm[1,0]:>6}  {cm[1,1]:>6}")

    print(f"\n   📋 CLASSIFICATION REPORT:")
    report = classification_report(y_test, y_pred, target_names=['Benign', 'Attack'])
    print(report)

    # Lưu classification report dạng dictionary
    report_dict = classification_report(y_test, y_pred, target_names=['Benign', 'Attack'], output_dict=True)

    # Thêm confusion matrix và các metrics khác vào results
    results['confusion_matrix'] = cm.tolist()
    results['classification_report'] = report_dict

    # Thêm các metrics chi tiết cho từng class
    results['benign_precision'] = float(report_dict['Benign']['precision'])
    results['benign_recall'] = float(report_dict['Benign']['recall'])
    results['benign_f1'] = float(report_dict['Benign']['f1-score'])
    results['attack_precision'] = float(report_dict['Attack']['precision'])
    results['attack_recall'] = float(report_dict['Attack']['recall'])
    results['attack_f1'] = float(report_dict['Attack']['f1-score'])

    # Tính thêm một số metrics bổ sung
    tn, fp, fn, tp = cm.ravel()
    results['true_negative'] = int(tn)
    results['false_positive'] = int(fp)
    results['false_negative'] = int(fn)
    results['true_positive'] = int(tp)
    results['specificity'] = float(tn / (tn + fp + 1e-7))  # True Negative Rate
    results['false_positive_rate'] = float(fp / (fp + tn + 1e-7))
    results['false_negative_rate'] = float(fn / (fn + tp + 1e-7))

    return results, y_pred, y_pred_prob


def save_model_and_results(model, history, results, training_time, model_dir, y_pred=None, y_pred_prob=None):
    """
    Lưu model và kết quả training

    Args:
        model: Trained model
        history: Training history
        results: Evaluation results
        training_time: Thời gian training (seconds)
        model_dir: Đường dẫn lưu
        y_pred: Predictions (optional)
        y_pred_prob: Prediction probabilities (optional)
    """
    print("\n" + "="*80)
    print("💾 ĐANG LƯU MODEL VÀ KẾT QUẢ...")
    print("="*80)

    model_dir = Path(model_dir)
    model_dir.mkdir(parents=True, exist_ok=True)

    # Lưu model cuối cùng
    final_model_path = model_dir / 'final_model.keras'
    model.save(final_model_path)
    print(f"   ✅ Final model: {final_model_path}")

    # Lưu model weights
    weights_path = model_dir / 'model_weights.weights.h5'
    model.save_weights(weights_path)
    print(f"   ✅ Model weights: {weights_path}")

    # Lưu training history
    history_dict = {key: [float(v) for v in values] for key, values in history.history.items()}
    with open(model_dir / 'training_history.json', 'w') as f:
        json.dump(history_dict, f, indent=4)
    print(f"   ✅ Training history: training_history.json")

    # Lưu kết quả đánh giá với thông tin bổ sung
    results['training_time_seconds'] = float(training_time)
    results['training_time_minutes'] = float(training_time / 60)
    results['epochs_trained'] = int(len(history.history['loss']))

    # Thêm best validation metrics
    results['best_val_loss'] = float(min(history.history['val_loss']))
    results['best_val_accuracy'] = float(max(history.history['val_accuracy']))
    results['best_val_precision'] = float(max(history.history['val_precision']))
    results['best_val_recall'] = float(max(history.history['val_recall']))

    # Tính best val F1-score
    val_precisions = history.history['val_precision']
    val_recalls = history.history['val_recall']
    val_f1_scores = [2 * (p * r) / (p + r + 1e-7) for p, r in zip(val_precisions, val_recalls)]
    results['best_val_f1_score'] = float(max(val_f1_scores))

    # Epoch nào đạt best val_loss
    results['best_val_loss_epoch'] = int(np.argmin(history.history['val_loss']) + 1)
    results['best_val_accuracy_epoch'] = int(np.argmax(history.history['val_accuracy']) + 1)

    with open(model_dir / 'evaluation_results.json', 'w') as f:
        json.dump(results, f, indent=4)
    print(f"   ✅ Evaluation results: evaluation_results.json")

    # Lưu predictions nếu có
    if y_pred is not None:
        np.save(model_dir / 'y_pred.npy', y_pred)
        print(f"   ✅ Predictions: y_pred.npy")

    if y_pred_prob is not None:
        np.save(model_dir / 'y_pred_prob.npy', y_pred_prob)
        print(f"   ✅ Prediction probabilities: y_pred_prob.npy")

    # Lưu cấu hình training
    config = {
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'learning_rate': LEARNING_RATE,
        'dropout_rate': DROPOUT_RATE,
        'patience': PATIENCE,
        'random_seed': RANDOM_SEED,
        'tensorflow_version': tf.__version__,
        'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    with open(model_dir / 'training_config.json', 'w') as f:
        json.dump(config, f, indent=4)
    print(f"   ✅ Training config: training_config.json")

    print(f"\n📁 Tất cả file được lưu tại: {model_dir}")


def plot_training_history(history, model_dir):
    """
    Vẽ biểu đồ training history

    Args:
        history: Training history
        model_dir: Đường dẫn lưu hình
    """
    try:
        import matplotlib.pyplot as plt

        model_dir = Path(model_dir)
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # 1. Loss
        axes[0, 0].plot(history.history['loss'], label='Train Loss')
        axes[0, 0].plot(history.history['val_loss'], label='Val Loss')
        axes[0, 0].set_title('Model Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)

        # 2. Accuracy
        axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy')
        axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy')
        axes[0, 1].set_title('Model Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        axes[0, 1].grid(True)

        # 3. Precision
        axes[1, 0].plot(history.history['precision'], label='Train Precision')
        axes[1, 0].plot(history.history['val_precision'], label='Val Precision')
        axes[1, 0].set_title('Model Precision')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Precision')
        axes[1, 0].legend()
        axes[1, 0].grid(True)

        # 4. Recall
        axes[1, 1].plot(history.history['recall'], label='Train Recall')
        axes[1, 1].plot(history.history['val_recall'], label='Val Recall')
        axes[1, 1].set_title('Model Recall')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Recall')
        axes[1, 1].legend()
        axes[1, 1].grid(True)

        plt.tight_layout()
        plt.savefig(model_dir / 'training_history.png', dpi=150)
        plt.close()
        print(f"   ✅ Training history plot: training_history.png")

    except ImportError:
        print("   ⚠️ matplotlib không có sẵn, bỏ qua việc vẽ biểu đồ")


def plot_confusion_matrix(cm, model_dir):
    """
    Vẽ confusion matrix dưới dạng heatmap

    Args:
        cm: Confusion matrix (numpy array hoặc list)
        model_dir: Đường dẫn lưu hình
    """
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns

        model_dir = Path(model_dir)

        # Convert to numpy array if needed
        if isinstance(cm, list):
            cm = np.array(cm)

        # Vẽ heatmap
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Benign', 'Attack'],
                   yticklabels=['Benign', 'Attack'],
                   cbar_kws={'label': 'Count'})
        plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
        plt.ylabel('Actual', fontsize=12)
        plt.xlabel('Predicted', fontsize=12)
        plt.tight_layout()
        plt.savefig(model_dir / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"   ✅ Confusion matrix plot: confusion_matrix.png")

        # Vẽ normalized confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                   xticklabels=['Benign', 'Attack'],
                   yticklabels=['Benign', 'Attack'],
                   cbar_kws={'label': 'Percentage'})
        plt.title('Normalized Confusion Matrix', fontsize=16, fontweight='bold')
        plt.ylabel('Actual', fontsize=12)
        plt.xlabel('Predicted', fontsize=12)
        plt.tight_layout()
        plt.savefig(model_dir / 'confusion_matrix_normalized.png', dpi=150, bbox_inches='tight')
        plt.close()
        print(f"   ✅ Normalized confusion matrix plot: confusion_matrix_normalized.png")

    except ImportError as e:
        print(f"   ⚠️ matplotlib/seaborn không có sẵn, bỏ qua việc vẽ confusion matrix: {e}")


def main():
    """Hàm chính để train model"""

    print("\n" + "="*80)
    print("🧠 HUẤN LUYỆN MÔ HÌNH CNN - PHÁT HIỆN LƯU LƯỢNG MẠNG BẤT THƯỜNG")
    print("   Binary Classification: Benign vs Attack")
    print("="*80)

    # Bước 1: Load dữ liệu
    X_train, X_val, X_test, y_train, y_val, y_test, class_weights = load_training_data(TRAINING_DATA_DIR)

    # Bước 2: Xây dựng mô hình
    input_shape = (X_train.shape[1], X_train.shape[2])  # (n_features, 1)
    model = build_cnn_model(input_shape)

    # Bước 3: Tạo callbacks
    callbacks = create_callbacks(MODEL_DIR, LOG_DIR)

    # Bước 4: Huấn luyện
    history, training_time = train_model(
        model, X_train, y_train, X_val, y_val, class_weights, callbacks
    )

    # Bước 5: Đánh giá
    results, y_pred, y_pred_prob = evaluate_model(model, X_test, y_test)

    # Bước 6: Lưu model và kết quả
    save_model_and_results(model, history, results, training_time, MODEL_DIR, y_pred, y_pred_prob)

    # Bước 7: Vẽ biểu đồ
    plot_training_history(history, MODEL_DIR)

    # Bước 8: Vẽ confusion matrix
    if 'confusion_matrix' in results:
        plot_confusion_matrix(results['confusion_matrix'], MODEL_DIR)

    print("\n" + "="*80)
    print("✅ HOÀN THÀNH HUẤN LUYỆN!")
    print(f"   Test Accuracy:  {results['test_accuracy']*100:.2f}%")
    print(f"   Test Precision: {results['test_precision']*100:.2f}%")
    print(f"   Test Recall:    {results['test_recall']*100:.2f}%")
    print(f"   Test F1-Score:  {results['test_f1_score']*100:.2f}%")
    print("="*80)

    return model, history, results


if __name__ == "__main__":
    model, history, results = main()



🖥️ THÔNG TIN HỆ THỐNG
TensorFlow version: 2.18.0
✅ GPU available: 2 GPU(s)
   - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
   - PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
🌐 Đang chạy trên KAGGLE

🧠 HUẤN LUYỆN MÔ HÌNH CNN - PHÁT HIỆN LƯU LƯỢNG MẠNG BẤT THƯỜNG
   Binary Classification: Benign vs Attack

📂 ĐANG LOAD DỮ LIỆU TRAINING...
   ✅ X_train: (2100000, 69, 1)
   ✅ X_val:   (300000, 69, 1)
   ✅ X_test:  (600000, 69, 1)
   ✅ y_train: (2100000,)
   ✅ y_val:   (300000,)
   ✅ y_test:  (600000,)

   ⚖️ Class weights loaded:
      Class 0 (Benign): 0.7143
      Class 1 (Attack): 1.6667

   📊 PHÂN BỐ DỮ LIỆU:
      Train: Benign=1,470,000 (70.0%), Attack=630,000 (30.0%)
      Val: Benign=210,000 (70.0%), Attack=90,000 (30.0%)
      Test: Benign=420,000 (70.0%), Attack=180,000 (30.0%)

🏗️ ĐANG XÂY DỰNG MÔ HÌNH CNN
   Input shape: (69, 1)

   📋 KIẾN TRÚC MÔ HÌNH:


Model: "CNN_Binary_Classification"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 69, 32)         │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_1 (MaxPooling1D)        │ (None, 34, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 34, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_2 (MaxPooling1D)        │ (None, 17, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 17, 64)         │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_3 (MaxPooling1D)        │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 8, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_4 (MaxPooling1D)        │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 4, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ maxpool_5 (MaxPooling1D)        │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm (BatchNormalization) │ (None, 2, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,233 (90.75 KB)

 Trainable params: 23,105 (90.25 KB)

 Non-trainable params: 128 (512.00 B)


📌 ĐANG CẤU HÌNH CALLBACKS...
   ✅ EarlyStopping: patience=10
   ✅ ModelCheckpoint: /kaggle/working/models/best_model.keras
   ✅ ReduceLROnPlateau: factor=0.5, patience=5
   ✅ TensorBoard: /kaggle/working/logs/20251228-121244

🚀 BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH
   Epochs: 50
   Batch size: 256
   Learning rate: 0.001
   Class weights: Có
Epoch 1/50
8204/8204 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9208 - loss: 0.2457 - precision: 0.8701 - recall: 0.8715
Epoch 1: val_loss improved from inf to 0.14378, saving model to /kaggle/working/models/best_model.keras
8204/8204 ━━━━━━━━━━━━━━━━━━━━ 55s 6ms/step - accuracy: 0.9208 - loss: 0.2457 - precision: 0.8701 - recall: 0.8715 - val_accuracy: 0.9604 - val_loss: 0.1438 - val_precision: 0.9612 - val_recall: 0.9046 - learning_rate: 0.0010
Epoch 2/50
8203/8204 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9619 - loss: 0.1650 - precision: 0.9625 - recall: 0.9085
Epoch 2: val_loss improved from 0.14378 to 0.12872, saving model to /kaggle/working/mo

In [5]:
!rm -rf /kaggle/working/models

In [ ]:
"""
======================================================================================
TIỀN XỬ LÝ DATASET CICIDS2018 CHO MÔ HÌNH CNN - PHÁT HIỆN LƯU LƯỢNG MẠNG IOT BẤT THƯỜNG
======================================================================================

Script này thực hiện các bước tiền xử lý dữ liệu:
1. Đọc từng file CSV theo chunks để tối ưu bộ nhớ
2. Loại bỏ các cột không cần thiết (IP, Port, Timestamp, Flow ID)
3. Xử lý missing values, NaN, Inf
4. Loại bỏ các hàng trùng lặp
5. Chuyển đổi nhãn sang dạng binary (Benign=0, Attack=1)
6. Chuẩn hóa dữ liệu bằng StandardScaler
7. Lưu dữ liệu đã xử lý sang định dạng nhanh (parquet/npy)

Có thể chạy trên cả Kaggle và Local
"""

import os
import numpy as np
import pandas as pd
import pickle
import json
import gc
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# THƯ VIỆN CHUẨN HÓA VÀ XỬ LÝ DỮ LIỆU
# ============================================================================
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Kiểm tra môi trường chạy (Kaggle hoặc Local)
IS_KAGGLE = os.path.exists('/kaggle/input')

# Progress bar
try:
    from tqdm import tqdm
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False
    print("⚠️  tqdm không có sẵn. Cài đặt bằng: pip install tqdm")
    tqdm = lambda x, **kwargs: x

# ============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================================
if IS_KAGGLE:
    # Đường dẫn trên Kaggle - thay đổi theo dataset của bạn
    DATA_DIR = "/kaggle/input/cicids2018"  # Thay đổi nếu tên dataset khác
    OUTPUT_DIR = "/kaggle/working/processed_data_cnn"
    print(" Đang chạy trên KAGGLE")
else:
    # Đường dẫn Local
    DATA_DIR = r"D:\PROJECT\Machine Learning\IOT\CICIDS2018-CSV"
    OUTPUT_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\processed_data_cnn"
    print(" Đang chạy trên LOCAL")

# ============================================================================
# CẤU HÌNH XỬ LÝ DỮ LIỆU
# ============================================================================

# Kích thước chunk khi đọc CSV (điều chỉnh theo RAM của máy)
CHUNK_SIZE = 300000  # 300k rows mỗi chunk

# Random state để tái tạo kết quả
RANDOM_STATE = 42

# Loại scaler: 'standard' (StandardScaler) hoặc 'minmax' (MinMaxScaler)
SCALER_TYPE = 'standard'

# ============================================================================
# CẤU HÌNH SAMPLE CÂN BẰNG
# ============================================================================
# Tổng số mẫu mong muốn (train + val + test)
TOTAL_SAMPLES = 3500000  # 3 triệu mẫu

# Tỷ lệ phần trăm cho mỗi class
BENIGN_RATIO = 0.70  # 70% Benign = 2,100,000 mẫu
ATTACK_RATIO = 0.30  # 30% Attack = 900,000 mẫu

# Tính số lượng mẫu cho mỗi class
TARGET_BENIGN = int(TOTAL_SAMPLES * BENIGN_RATIO)  # 2,100,000
TARGET_ATTACK = int(TOTAL_SAMPLES * ATTACK_RATIO)  # 900,000

# ============================================================================
# DANH SÁCH CÁC CỘT CẦN LOẠI BỎ
# ============================================================================

# Các cột không cần thiết cho việc huấn luyện CNN
COLUMNS_TO_DROP = [
    # Thông tin định danh - không mang tính tổng quát
    'Flow ID',          # ID duy nhất cho mỗi flow
    'Src IP',           # IP nguồn
    'Dst IP',           # IP đích
    'Src Port',         # Port nguồn
    'Timestamp',        # Thời gian - không liên quan đến pattern

    # Các cột flag không mang nhiều thông tin
    # 'Bwd PSH Flags',    # Hiếm khi có giá trị khác 0
    # 'Bwd URG Flags',    # Hiếm khi có giá trị khác 0
    # 'Fwd URG Flags',    # Hiếm khi có giá trị khác 0
]

# Cột nhãn
LABEL_COLUMN = 'Label'

# ============================================================================
# CLASS XỬ LÝ DỮ LIỆU CHO CNN
# ============================================================================

class CICIDS2018_CNN_Preprocessor:
    """
    Class xử lý dữ liệu CICIDS2018 cho mô hình CNN phát hiện bất thường

    Các bước xử lý:
    1. Đọc dữ liệu theo chunks
    2. Loại bỏ cột không cần thiết
    3. Xử lý giá trị thiếu, NaN, Inf
    4. Loại bỏ duplicate
    5. Chuyển đổi nhãn sang binary
    6. Chuẩn hóa features
    7. Lưu dữ liệu đã xử lý
    """

    def __init__(self, data_dir, output_dir, chunk_size=CHUNK_SIZE,
                 scaler_type=SCALER_TYPE, target_benign=TARGET_BENIGN,
                 target_attack=TARGET_ATTACK):
        """
        Khởi tạo preprocessor

        Args:
            data_dir: Đường dẫn thư mục chứa file CSV
            output_dir: Đường dẫn thư mục lưu kết quả
            chunk_size: Số dòng mỗi chunk khi đọc CSV
            scaler_type: Loại scaler ('standard' hoặc 'minmax')
            target_benign: Số lượng mẫu Benign mong muốn
            target_attack: Số lượng mẫu Attack mong muốn
        """
        self.data_dir = Path(data_dir)
        self.output_dir = Path(output_dir)
        self.chunk_size = chunk_size
        self.scaler_type = scaler_type
        self.target_benign = target_benign
        self.target_attack = target_attack

        # Khởi tạo scaler
        if scaler_type == 'minmax':
            self.scaler = MinMaxScaler()
        else:
            self.scaler = StandardScaler()

        # Tạo thư mục output nếu chưa có
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # Thống kê
        self.stats = {
            'total_rows_read': 0,
            'rows_after_cleaning': 0,
            'duplicates_removed': 0,
            'nan_inf_replaced': 0,
            'benign_count': 0,
            'attack_count': 0,
            'feature_count': 0,
            'processing_time': 0
        }

        # Lưu tên các features
        self.feature_names = None

    def _get_csv_files(self):
        """Lấy danh sách các file CSV trong thư mục data"""
        csv_files = list(self.data_dir.glob("*_TrafficForML_CICFlowMeter.csv"))
        if not csv_files:
            # Thử pattern khác cho Kaggle
            csv_files = list(self.data_dir.glob("*.csv"))

        if not csv_files:
            raise FileNotFoundError(f"Không tìm thấy file CSV trong {self.data_dir}")

        print(f"\n📂 Tìm thấy {len(csv_files)} file CSV:")
        for f in sorted(csv_files):
            print(f"   - {f.name}")
        return sorted(csv_files)

    def _clean_column_names(self, df):
        """Chuẩn hóa tên cột (loại bỏ khoảng trắng thừa)"""
        df.columns = df.columns.str.strip()
        return df

    def _drop_unnecessary_columns(self, df):
        """Loại bỏ các cột không cần thiết cho huấn luyện"""
        columns_to_drop = [col for col in COLUMNS_TO_DROP if col in df.columns]

        if columns_to_drop:
            df = df.drop(columns=columns_to_drop)

        return df

    def _convert_to_numeric(self, df):
        """Chuyển đổi các cột về dạng số"""
        # Lấy tất cả cột trừ Label
        feature_cols = [col for col in df.columns if col != LABEL_COLUMN]

        for col in feature_cols:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col], errors='coerce')

        return df

    def _handle_nan_inf(self, df):
        """Xử lý giá trị NaN và Infinity"""
        feature_cols = [col for col in df.columns if col != LABEL_COLUMN]

        # Đếm số lượng NaN và Inf trước khi xử lý
        nan_count = df[feature_cols].isna().sum().sum()
        inf_count = np.isinf(df[feature_cols].select_dtypes(include=[np.number])).sum().sum()

        self.stats['nan_inf_replaced'] += nan_count + inf_count

        # Thay thế Infinity bằng NaN trước
        df[feature_cols] = df[feature_cols].replace([np.inf, -np.inf], np.nan)

        # Thay thế NaN bằng 0 (hoặc có thể dùng median/mean)
        df[feature_cols] = df[feature_cols].fillna(0)

        return df

    def _remove_duplicates(self, df):
        """Loại bỏ các hàng trùng lặp"""
        rows_before = len(df)
        df = df.drop_duplicates()
        rows_after = len(df)

        self.stats['duplicates_removed'] += (rows_before - rows_after)

        return df

    def _convert_to_binary_label(self, df):
        """
        Chuyển đổi nhãn sang dạng binary:
        - Benign -> 0 (lưu lượng bình thường)
        - Tất cả các loại tấn công khác -> 1 (lưu lượng bất thường)
        """
        if LABEL_COLUMN not in df.columns:
            raise ValueError(f"Không tìm thấy cột '{LABEL_COLUMN}' trong dữ liệu")

        # Chuẩn hóa nhãn (loại bỏ khoảng trắng, lowercase)
        df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(str).str.strip().str.lower()

        # Loại bỏ các hàng có nhãn là 'label' (header bị lẫn vào data)
        df = df[df[LABEL_COLUMN] != 'label']

        # Chuyển đổi sang binary: Benign=0, Attack=1
        df['binary_label'] = (df[LABEL_COLUMN] != 'benign').astype(int)

        # Đếm số lượng mỗi class
        benign_count = (df['binary_label'] == 0).sum()
        attack_count = (df['binary_label'] == 1).sum()

        self.stats['benign_count'] += benign_count
        self.stats['attack_count'] += attack_count

        # Xóa cột Label gốc, giữ lại binary_label
        df = df.drop(columns=[LABEL_COLUMN])

        return df

    def _process_single_file(self, csv_file):
        """
        Xử lý một file CSV theo chunks

        Args:
            csv_file: Đường dẫn file CSV

        Returns:
            DataFrame đã được xử lý
        """
        print(f"\n📄 Đang xử lý: {csv_file.name}")

        processed_chunks = []
        chunk_iterator = pd.read_csv(csv_file, chunksize=self.chunk_size,
                                     low_memory=False, encoding='utf-8')

        # Progress bar cho chunks
        if TQDM_AVAILABLE:
            # Ước tính số chunks dựa trên file size
            file_size = csv_file.stat().st_size
            estimated_chunks = max(1, file_size // (self.chunk_size * 500))  # Ước tính
            chunk_iterator = tqdm(chunk_iterator, desc="   Chunks",
                                  total=estimated_chunks, unit="chunk")

        for chunk in chunk_iterator:
            self.stats['total_rows_read'] += len(chunk)

            # Bước 1: Chuẩn hóa tên cột
            chunk = self._clean_column_names(chunk)

            # Bước 2: Loại bỏ cột không cần thiết
            chunk = self._drop_unnecessary_columns(chunk)

            # Bước 3: Chuyển đổi sang dạng số
            chunk = self._convert_to_numeric(chunk)

            # Bước 4: Xử lý NaN và Inf
            chunk = self._handle_nan_inf(chunk)

            # Bước 5: Chuyển đổi nhãn sang binary
            chunk = self._convert_to_binary_label(chunk)

            processed_chunks.append(chunk)

            # Giải phóng bộ nhớ
            gc.collect()

        # Gộp các chunks lại
        if processed_chunks:
            df = pd.concat(processed_chunks, ignore_index=True)
            del processed_chunks
            gc.collect()
            return df

        return None

    def process_all_files(self):
        """
        Xử lý tất cả các file CSV và gộp lại

        Returns:
            DataFrame đã xử lý hoàn chỉnh
        """
        start_time = datetime.now()
        print("\n" + "="*80)
        print("🚀 BẮT ĐẦU XỬ LÝ DỮ LIỆU CICIDS2018 CHO CNN")
        print("="*80)

        csv_files = self._get_csv_files()

        all_dataframes = []

        # Xử lý từng file
        for csv_file in csv_files:
            df = self._process_single_file(csv_file)
            if df is not None:
                all_dataframes.append(df)
                print(f"   ✅ Đã xử lý: {len(df):,} mẫu")

        # Gộp tất cả lại
        print("\n" + "-"*80)
        print("📊 ĐANG GỘP VÀ XỬ LÝ CUỐI CÙNG...")

        df_combined = pd.concat(all_dataframes, ignore_index=True)
        del all_dataframes
        gc.collect()

        print(f"   Tổng số mẫu sau khi gộp: {len(df_combined):,}")

        # Loại bỏ duplicate trên toàn bộ dataset
        print("   Đang loại bỏ duplicate...")
        df_combined = self._remove_duplicates(df_combined)
        print(f"   Số mẫu sau khi loại duplicate: {len(df_combined):,}")

        # Cập nhật thống kê
        self.stats['rows_after_cleaning'] = len(df_combined)
        self.stats['feature_count'] = len(df_combined.columns) - 1  # Trừ cột label

        # Lưu tên features
        self.feature_names = [col for col in df_combined.columns if col != 'binary_label']

        end_time = datetime.now()
        self.stats['processing_time'] = (end_time - start_time).total_seconds()

        return df_combined

    def balanced_sample(self, df):
        """
        Sample dữ liệu với số lượng cân bằng theo target đã định

        Lấy chính xác:
        - TARGET_BENIGN mẫu Benign (2,100,000)
        - TARGET_ATTACK mẫu Attack (900,000)

        Args:
            df: DataFrame đã clean

        Returns:
            DataFrame đã được sample cân bằng
        """
        print("\n" + "="*80)
        print("⚖️ ĐANG SAMPLE CÂN BẰNG DỮ LIỆU")
        print("="*80)

        # Tách theo class
        df_benign = df[df['binary_label'] == 0]
        df_attack = df[df['binary_label'] == 1]

        n_benign = len(df_benign)
        n_attack = len(df_attack)

        print(f"\n   Dữ liệu gốc (sau khi clean):")
        print(f"   - Benign: {n_benign:,}")
        print(f"   - Attack: {n_attack:,}")
        print(f"   - Tổng: {n_benign + n_attack:,}")

        print(f"\n   Target mong muốn:")
        print(f"   - Benign: {self.target_benign:,} ({BENIGN_RATIO*100:.0f}%)")
        print(f"   - Attack: {self.target_attack:,} ({ATTACK_RATIO*100:.0f}%)")
        print(f"   - Tổng: {self.target_benign + self.target_attack:,}")

        # Kiểm tra và điều chỉnh nếu không đủ mẫu
        actual_benign = min(self.target_benign, n_benign)
        actual_attack = min(self.target_attack, n_attack)

        if actual_benign < self.target_benign:
            print(f"\n   ⚠️ Không đủ Benign! Chỉ có {n_benign:,}, cần {self.target_benign:,}")
        if actual_attack < self.target_attack:
            print(f"\n   ⚠️ Không đủ Attack! Chỉ có {n_attack:,}, cần {self.target_attack:,}")

        # Random sample từ mỗi class
        print(f"\n   Đang sample...")
        df_benign_sampled = df_benign.sample(n=actual_benign, random_state=RANDOM_STATE)
        df_attack_sampled = df_attack.sample(n=actual_attack, random_state=RANDOM_STATE)

        # Gộp lại và shuffle
        df_balanced = pd.concat([df_benign_sampled, df_attack_sampled], ignore_index=True)
        df_balanced = df_balanced.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

        # Thống kê kết quả
        final_benign = (df_balanced['binary_label'] == 0).sum()
        final_attack = (df_balanced['binary_label'] == 1).sum()
        total = len(df_balanced)

        print(f"\n   ✅ Kết quả sau khi sample:")
        print(f"   - Benign: {final_benign:,} ({final_benign/total*100:.1f}%)")
        print(f"   - Attack: {final_attack:,} ({final_attack/total*100:.1f}%)")
        print(f"   - Tổng: {total:,}")
        print(f"   - Tỷ lệ Benign:Attack = {final_benign/final_attack:.2f}:1")

        # Cập nhật stats
        self.stats['benign_count'] = final_benign
        self.stats['attack_count'] = final_attack
        self.stats['rows_after_cleaning'] = total

        return df_balanced

    def normalize_features(self, df):
        """
        Chuẩn hóa các features bằng scaler

        Args:
            df: DataFrame chứa features và label

        Returns:
            X_normalized: Features đã chuẩn hóa
            y: Labels
        """
        print("\n🔄 ĐANG CHUẨN HÓA DỮ LIỆU...")

        # Tách features và label
        X = df.drop(columns=['binary_label']).values
        y = df['binary_label'].values

        # Chuẩn hóa features
        X_normalized = self.scaler.fit_transform(X)

        print(f"   Scaler type: {self.scaler_type}")
        print(f"   Shape X: {X_normalized.shape}")
        print(f"   Shape y: {y.shape}")

        return X_normalized, y

    def reshape_for_cnn(self, X):
        """
        Reshape dữ liệu cho CNN 1D

        CNN 1D yêu cầu input shape: (samples, features, channels)
        Trong trường hợp này: (samples, n_features, 1)

        Args:
            X: Features đã chuẩn hóa, shape (samples, features)

        Returns:
            X_reshaped: Shape (samples, features, 1)
        """
        print("\n🔄 ĐANG RESHAPE DỮ LIỆU CHO CNN...")

        X_reshaped = X.reshape(X.shape[0], X.shape[1], 1)

        print(f"   Shape sau reshape: {X_reshaped.shape}")

        return X_reshaped

    def split_data(self, X, y, test_size=0.2, val_size=0.1):
        """
        Chia dữ liệu thành train/val/test sets

        Args:
            X: Features
            y: Labels
            test_size: Tỷ lệ test set
            val_size: Tỷ lệ validation set (từ train)

        Returns:
            X_train, X_val, X_test, y_train, y_val, y_test
        """
        print("\n📊 ĐANG CHIA DỮ LIỆU TRAIN/VAL/TEST...")

        # Chia train+val / test
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=test_size, random_state=RANDOM_STATE, stratify=y
        )

        # Chia train / val
        val_ratio = val_size / (1 - test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_ratio, random_state=RANDOM_STATE, stratify=y_temp
        )

        print(f"   Train set: {X_train.shape[0]:,} mẫu")
        print(f"   Val set:   {X_val.shape[0]:,} mẫu")
        print(f"   Test set:  {X_test.shape[0]:,} mẫu")

        # Thống kê phân bố class
        print(f"\n   Phân bố Train - Benign: {(y_train==0).sum():,}, Attack: {(y_train==1).sum():,}")
        print(f"   Phân bố Val   - Benign: {(y_val==0).sum():,}, Attack: {(y_val==1).sum():,}")
        print(f"   Phân bố Test  - Benign: {(y_test==0).sum():,}, Attack: {(y_test==1).sum():,}")

        return X_train, X_val, X_test, y_train, y_val, y_test


    def save_processed_data(self, X_train, X_val, X_test, y_train, y_val, y_test):
        """
        Lưu dữ liệu đã xử lý sang định dạng nhanh

        Lưu thành các file:
        - X_train.npy, X_val.npy, X_test.npy
        - y_train.npy, y_val.npy, y_test.npy
        - scaler.pkl
        - metadata.json
        """
        print("\n ĐANG LƯU DỮ LIỆU ĐÃ XỬ LÝ...")

        # Lưu numpy arrays
        np.save(self.output_dir / 'X_train.npy', X_train)
        np.save(self.output_dir / 'X_val.npy', X_val)
        np.save(self.output_dir / 'X_test.npy', X_test)
        np.save(self.output_dir / 'y_train.npy', y_train)
        np.save(self.output_dir / 'y_val.npy', y_val)
        np.save(self.output_dir / 'y_test.npy', y_test)

        print(f"   ✅ Đã lưu X_train.npy: {X_train.shape}")
        print(f"   ✅ Đã lưu X_val.npy: {X_val.shape}")
        print(f"   ✅ Đã lưu X_test.npy: {X_test.shape}")
        print(f"   ✅ Đã lưu y_train.npy: {y_train.shape}")
        print(f"   ✅ Đã lưu y_val.npy: {y_val.shape}")
        print(f"   ✅ Đã lưu y_test.npy: {y_test.shape}")

        # Lưu scaler
        with open(self.output_dir / 'scaler.pkl', 'wb') as f:
            pickle.dump(self.scaler, f)
        print(f"   ✅ Đã lưu scaler.pkl")

        # Lưu feature names
        with open(self.output_dir / 'feature_names.txt', 'w') as f:
            for name in self.feature_names:
                f.write(name + '\n')
        print(f"   ✅ Đã lưu feature_names.txt")

        # Chuyển đổi stats sang kiểu Python native (để tránh lỗi JSON với numpy.int64)
        stats_native = {}
        for key, value in self.stats.items():
            if hasattr(value, 'item'):  # Kiểm tra nếu là numpy type
                stats_native[key] = value.item()
            elif isinstance(value, (np.integer, np.floating)):
                stats_native[key] = int(value) if isinstance(value, np.integer) else float(value)
            else:
                stats_native[key] = value

        # Lưu metadata
        metadata = {
            'n_features': len(self.feature_names),
            'feature_names': self.feature_names,
            'train_samples': int(X_train.shape[0]),
            'val_samples': int(X_val.shape[0]),
            'test_samples': int(X_test.shape[0]),
            'input_shape': [int(x) for x in X_train.shape[1:]],
            'scaler_type': self.scaler_type,
            'stats': stats_native,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        with open(self.output_dir / 'metadata.json', 'w') as f:
            json.dump(metadata, f, indent=4)
        print(f"   ✅ Đã lưu metadata.json")

        print(f"\n📁 Tất cả file được lưu tại: {self.output_dir}")

    def print_summary(self):
        """In tóm tắt quá trình xử lý"""
        print("\n" + "="*80)
        print("📊 TÓM TẮT XỬ LÝ DỮ LIỆU")
        print("="*80)
        print(f"   Tổng số dòng đọc được:     {self.stats['total_rows_read']:,}")
        print(f"   Số dòng sau khi xử lý:     {self.stats['rows_after_cleaning']:,}")
        print(f"   Số duplicate đã loại:      {self.stats['duplicates_removed']:,}")
        print(f"   Số NaN/Inf đã thay thế:    {self.stats['nan_inf_replaced']:,}")
        print(f"   Số features:               {self.stats['feature_count']}")
        print(f"   Số mẫu Benign:             {self.stats['benign_count']:,}")
        print(f"   Số mẫu Attack:             {self.stats['attack_count']:,}")
        print(f"   Thời gian xử lý:           {self.stats['processing_time']:.2f} giây")
        print("="*80)


def main():
    """Hàm chính để chạy preprocessing"""

    print("\n" + "="*80)
    print("🔧 TIỀN XỬ LÝ DỮ LIỆU CICIDS2018 CHO MÔ HÌNH CNN")
    print("   Phát hiện lưu lượng mạng IoT bất thường")
    print("="*80)

    print(f"\n📋 CẤU HÌNH:")
    print(f"   - Tổng mẫu mong muốn: {TOTAL_SAMPLES:,}")
    print(f"   - Benign: {TARGET_BENIGN:,} ({BENIGN_RATIO*100:.0f}%)")
    print(f"   - Attack: {TARGET_ATTACK:,} ({ATTACK_RATIO*100:.0f}%)")

    # Khởi tạo preprocessor
    preprocessor = CICIDS2018_CNN_Preprocessor(
        data_dir=DATA_DIR,
        output_dir=OUTPUT_DIR,
        chunk_size=CHUNK_SIZE,
        scaler_type=SCALER_TYPE,
        target_benign=TARGET_BENIGN,
        target_attack=TARGET_ATTACK
    )

    # Bước 1: Xử lý tất cả các file CSV (clean data)
    df = preprocessor.process_all_files()

    # Bước 2: SAMPLE CÂN BẰNG TRƯỚC KHI CHIA
    # Điều này đảm bảo train/val/test đều có tỷ lệ 70-30
    df = preprocessor.balanced_sample(df)

    # Bước 3: Chuẩn hóa features
    X, y = preprocessor.normalize_features(df)

    # Giải phóng bộ nhớ của DataFrame
    del df
    gc.collect()

    # Bước 4: Reshape cho CNN
    X = preprocessor.reshape_for_cnn(X)

    # Bước 5: Chia dữ liệu (stratify để giữ tỷ lệ 70-30 trong tất cả các tập)
    X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.split_data(X, y)

    # Giải phóng bộ nhớ
    del X, y
    gc.collect()

    # Bước 6: Lưu dữ liệu
    preprocessor.save_processed_data(X_train, X_val, X_test, y_train, y_val, y_test)

    # In tóm tắt
    preprocessor.print_summary()

    print("\n✅ HOÀN THÀNH! Dữ liệu đã sẵn sàng cho việc huấn luyện CNN.")

    return preprocessor


if __name__ == "__main__":
    preprocessor = main()



In [7]:
import shutil
import os

# Đường dẫn chuẩn trong Kaggle
directory_to_zip = '/kaggle/working/models'
output_filename = 'model_cnn_5layer' # Tên file kết quả (không cần đuôi .zip)

# Kiểm tra xem có file nào trong đó không
if os.path.exists(directory_to_zip):
    print("Đang nén dữ liệu...")
    # Nén thư mục
    shutil.make_archive(output_filename, 'zip', directory_to_zip)
    print(f"✅ Xong! File nằm tại: {os.path.join(directory_to_zip, output_filename + '.zip')}")
else:
    print("Không tìm thấy thư mục working.")

Đang nén dữ liệu...
✅ Xong! File nằm tại: /kaggle/working/models/model_cnn_5layer.zip


In [ ]:
"""
======================================================================================
HUẤN LUYỆN MÔ HÌNH CNN - PHÁT HIỆN LƯU LƯỢNG MẠNG IOT BẤT THƯỜNG
======================================================================================

Script này huấn luyện mô hình CNN 1D cho bài toán phân loại binary:
- Benign (0): Lưu lượng mạng bình thường
- Attack (1): Lưu lượng mạng bất thường/tấn công

Mô hình CNN 1D được thiết kế để học các patterns từ network flow features.

Có thể chạy trên cả Kaggle và Local.
"""

import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# KIỂM TRA MÔI TRƯỜNG VÀ IMPORT THƯ VIỆN
# ============================================================================

# Kiểm tra môi trường chạy (Kaggle hoặc Local)
IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    print("🌐 Đang chạy trên KAGGLE")
else:
    print("💻 Đang chạy trên LOCAL")

# Import TensorFlow/Keras
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, models, callbacks
    from tensorflow.keras.optimizers import Adam
    print(f"✅ TensorFlow version: {tf.__version__}")
except ImportError:
    print("❌ Lỗi: TensorFlow chưa được cài đặt!")
    print("   Cài đặt bằng: pip install tensorflow")
    sys.exit(1)

# Import sklearn cho metrics
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)

# ============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ============================================================================

if IS_KAGGLE:
    # Đường dẫn trên Kaggle
    PROCESSED_DATA_DIR = "/kaggle/working/processed_data_cnn"
    OUTPUT_DIR = "/kaggle/working/cnn_results"
else:
    # Đường dẫn Local
    PROCESSED_DATA_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\processed_data_cnn"
    OUTPUT_DIR = r"D:\PROJECT\Machine Learning\IOT\CNN\results"

# ============================================================================
# CẤU HÌNH MÔ HÌNH VÀ HUẤN LUYỆN
# ============================================================================

# Hyperparameters cho CNN
CNN_CONFIG = {
    # Kiến trúc mạng
    'conv_filters': [64, 128, 256],      # Số filters cho mỗi Conv layer
    'kernel_size': 3,                     # Kích thước kernel
    'pool_size': 2,                       # Kích thước pooling
    'dense_units': [128, 64],             # Số units cho Dense layers
    'dropout_rate': 0.3,                  # Tỷ lệ dropout

    # Huấn luyện
    'batch_size': 256,                    # Batch size
    'epochs': 30,                         # Số epochs tối đa
    'learning_rate': 0.001,               # Learning rate
    'early_stopping_patience': 10,        # Patience cho early stopping
    'reduce_lr_patience': 5,              # Patience cho reduce LR
    'reduce_lr_factor': 0.5,              # Factor giảm LR
    'min_lr': 1e-7,                       # LR tối thiểu

    # Class weights để xử lý imbalanced data
    'use_class_weight': True,             # Sử dụng class weight
}

# ============================================================================
# LOAD DỮ LIỆU ĐÃ XỬ LÝ
# ============================================================================

def load_processed_data(data_dir):
    """
    Load dữ liệu đã được tiền xử lý

    Args:
        data_dir: Đường dẫn thư mục chứa dữ liệu

    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test, metadata
    """
    data_dir = Path(data_dir)

    print("\n" + "="*80)
    print("📂 ĐANG LOAD DỮ LIỆU ĐÃ XỬ LÝ")
    print("="*80)

    # Load numpy arrays
    X_train = np.load(data_dir / 'X_train.npy')
    X_val = np.load(data_dir / 'X_val.npy')
    X_test = np.load(data_dir / 'X_test.npy')
    y_train = np.load(data_dir / 'y_train.npy')
    y_val = np.load(data_dir / 'y_val.npy')
    y_test = np.load(data_dir / 'y_test.npy')

    print(f"   X_train: {X_train.shape}")
    print(f"   X_val:   {X_val.shape}")
    print(f"   X_test:  {X_test.shape}")
    print(f"   y_train: {y_train.shape}")
    print(f"   y_val:   {y_val.shape}")
    print(f"   y_test:  {y_test.shape}")

    # Load metadata
    with open(data_dir / 'metadata.json', 'r') as f:
        metadata = json.load(f)

    print(f"\n   Số features: {metadata['n_features']}")
    print(f"   Train samples: {metadata['train_samples']:,}")
    print(f"   Val samples: {metadata['val_samples']:,}")
    print(f"   Test samples: {metadata['test_samples']:,}")

    return X_train, X_val, X_test, y_train, y_val, y_test, metadata


# ============================================================================
# XÂY DỰNG MÔ HÌNH CNN
# ============================================================================

def build_cnn_model(input_shape, config=CNN_CONFIG):
    """
    Xây dựng mô hình CNN 1D cho phân loại binary

    Kiến trúc:
    - Nhiều block Conv1D + BatchNorm + ReLU + MaxPooling + Dropout
    - Flatten
    - Dense layers với Dropout
    - Output layer với Sigmoid

    Args:
        input_shape: Shape của input (n_features, 1)
        config: Dictionary chứa các hyperparameters

    Returns:
        model: Mô hình Keras đã compile
    """
    print("\n" + "="*80)
    print("🏗️ ĐANG XÂY DỰNG MÔ HÌNH CNN")
    print("="*80)

    model = models.Sequential(name="CNN_IDS")

    # Input layer
    model.add(layers.InputLayer(input_shape=input_shape))

    # Convolutional blocks
    for i, filters in enumerate(config['conv_filters']):
        # Conv1D layer
        model.add(layers.Conv1D(
            filters=filters,
            kernel_size=config['kernel_size'],
            padding='same',
            name=f'conv1d_{i+1}'
        ))

        # Batch Normalization
        model.add(layers.BatchNormalization(name=f'bn_{i+1}'))

        # Activation
        model.add(layers.Activation('relu', name=f'relu_{i+1}'))

        # MaxPooling (chỉ áp dụng nếu kích thước đủ lớn)
        model.add(layers.MaxPooling1D(
            pool_size=config['pool_size'],
            padding='same',
            name=f'maxpool_{i+1}'
        ))

        # Dropout
        model.add(layers.Dropout(
            config['dropout_rate'],
            name=f'dropout_conv_{i+1}'
        ))

    # Flatten
    model.add(layers.Flatten(name='flatten'))

    # Dense layers
    for i, units in enumerate(config['dense_units']):
        model.add(layers.Dense(units, name=f'dense_{i+1}'))
        model.add(layers.BatchNormalization(name=f'bn_dense_{i+1}'))
        model.add(layers.Activation('relu', name=f'relu_dense_{i+1}'))
        model.add(layers.Dropout(config['dropout_rate'], name=f'dropout_dense_{i+1}'))

    # Output layer (binary classification)
    model.add(layers.Dense(1, activation='sigmoid', name='output'))

    # Compile model
    optimizer = Adam(learning_rate=config['learning_rate'])

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )

    # In summary
    print(f"\n   Input shape: {input_shape}")
    print(f"   Conv filters: {config['conv_filters']}")
    print(f"   Dense units: {config['dense_units']}")
    print(f"   Dropout rate: {config['dropout_rate']}")
    print(f"   Learning rate: {config['learning_rate']}")

    model.summary()

    return model


# ============================================================================
# TÍNH CLASS WEIGHTS
# ============================================================================

def compute_class_weights(y_train):
    """
    Tính class weights để xử lý imbalanced data

    Args:
        y_train: Labels của training set

    Returns:
        class_weight: Dictionary {0: weight_0, 1: weight_1}
    """
    # Đếm số lượng mỗi class
    n_benign = (y_train == 0).sum()
    n_attack = (y_train == 1).sum()
    total = len(y_train)

    # Tính weights
    weight_benign = total / (2 * n_benign)
    weight_attack = total / (2 * n_attack)

    class_weight = {
        0: weight_benign,
        1: weight_attack
    }

    print(f"\n📊 Class weights:")
    print(f"   Benign (0): {n_benign:,} mẫu, weight = {weight_benign:.4f}")
    print(f"   Attack (1): {n_attack:,} mẫu, weight = {weight_attack:.4f}")

    return class_weight


# ============================================================================
# CALLBACKS
# ============================================================================

def get_callbacks(output_dir, config=CNN_CONFIG):
    """
    Tạo các callbacks cho quá trình huấn luyện

    Args:
        output_dir: Đường dẫn lưu model
        config: Dictionary chứa các hyperparameters

    Returns:
        List các callbacks
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    callback_list = [
        # Early stopping - dừng nếu val_loss không cải thiện
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=config['early_stopping_patience'],
            restore_best_weights=True,
            verbose=1
        ),

        # Reduce learning rate khi val_loss không cải thiện
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=config['reduce_lr_factor'],
            patience=config['reduce_lr_patience'],
            min_lr=config['min_lr'],
            verbose=1
        ),

        # Model checkpoint - lưu model tốt nhất
        callbacks.ModelCheckpoint(
            filepath=str(output_dir / 'best_model.keras'),
            monitor='val_auc',
            mode='max',
            save_best_only=True,
            verbose=1
        ),

        # TensorBoard logs (optional)
        callbacks.TensorBoard(
            log_dir=str(output_dir / 'logs'),
            histogram_freq=1
        ),

        # CSV logger - lưu history ra file
        callbacks.CSVLogger(
            filename=str(output_dir / 'training_history.csv'),
            separator=',',
            append=False
        )
    ]

    return callback_list


# ============================================================================
# HUẤN LUYỆN MÔ HÌNH
# ============================================================================

def train_model(model, X_train, y_train, X_val, y_val,
                output_dir, config=None):
    """
    Huấn luyện mô hình CNN

    Args:
        model: Mô hình Keras
        X_train, y_train: Dữ liệu training
        X_val, y_val: Dữ liệu validation
        output_dir: Đường dẫn lưu kết quả
        config: Dictionary chứa các hyperparameters

    Returns:
        history: History object của quá trình training
    """
    if config is None:
        config = CNN_CONFIG
    print("\n" + "="*80)
    print("🚀 BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH")
    print("="*80)

    # Tính class weights nếu cần
    class_weight = None
    if config['use_class_weight']:
        class_weight = compute_class_weights(y_train)

    # Lấy callbacks
    callback_list = get_callbacks(output_dir, config)

    print(f"\n   Batch size: {config['batch_size']}")
    print(f"   Max epochs: {config['epochs']}")
    print(f"   Early stopping patience: {config['early_stopping_patience']}")

    # Huấn luyện
    start_time = datetime.now()

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        class_weight=class_weight,
        callbacks=callback_list,
        verbose=1
    )

    end_time = datetime.now()
    training_time = (end_time - start_time).total_seconds()

    print(f"\n⏱️ Thời gian huấn luyện: {training_time/60:.2f} phút")

    return history, training_time


# ============================================================================
# ĐÁNH GIÁ MÔ HÌNH
# ============================================================================

def evaluate_model(model, X_test, y_test):
    """
    Đánh giá mô hình trên test set

    Args:
        model: Mô hình đã train
        X_test, y_test: Dữ liệu test

    Returns:
        results: Dictionary chứa các metrics
    """
    print("\n" + "="*80)
    print("📊 ĐÁNH GIÁ MÔ HÌNH TRÊN TEST SET")
    print("="*80)

    # Dự đoán
    y_pred_prob = model.predict(X_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    # Tính các metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)

    # In kết quả
    print(f"\n   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC-ROC:   {auc:.4f}")

    # Classification report
    print("\n" + "-"*60)
    print("CLASSIFICATION REPORT:")
    print("-"*60)
    target_names = ['Benign', 'Attack']
    print(classification_report(y_test, y_pred, target_names=target_names))

    # Confusion matrix
    print("-"*60)
    print("CONFUSION MATRIX:")
    print("-"*60)
    cm = confusion_matrix(y_test, y_pred)
    print(f"                Predicted")
    print(f"              Benign  Attack")
    print(f"Actual Benign   {cm[0,0]:6d}  {cm[0,1]:6d}")
    print(f"       Attack   {cm[1,0]:6d}  {cm[1,1]:6d}")

    # Tính detection rate và false alarm rate
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0

    print(f"\n   Detection Rate (True Positive Rate): {detection_rate:.4f}")
    print(f"   False Alarm Rate (False Positive Rate): {false_alarm_rate:.4f}")

    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc,
        'detection_rate': detection_rate,
        'false_alarm_rate': false_alarm_rate,
        'confusion_matrix': cm.tolist(),
        'y_pred_prob': y_pred_prob,
        'y_pred': y_pred
    }

    return results


# ============================================================================
# VẼ BIỂU ĐỒ
# ============================================================================

def plot_training_history(history, output_dir):
    """
    Vẽ biểu đồ training history

    Args:
        history: History object từ model.fit()
        output_dir: Đường dẫn lưu biểu đồ
    """
    output_dir = Path(output_dir)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Plot Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title('Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 1].set_title('Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Plot Precision & Recall
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 0].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 0].set_title('Precision & Recall', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Plot AUC
    axes[1, 1].plot(history.history['auc'], label='Train AUC', linewidth=2)
    axes[1, 1].plot(history.history['val_auc'], label='Val AUC', linewidth=2)
    axes[1, 1].set_title('AUC-ROC', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('AUC')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.suptitle('Training History - CNN IoT Anomaly Detection',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(output_dir / 'training_history.png', dpi=150, bbox_inches='tight')
    plt.close()

    print(f"\n📊 Đã lưu biểu đồ training history: {output_dir / 'training_history.png'}")


def plot_confusion_matrix(cm, output_dir):
    """
    Vẽ confusion matrix

    Args:
        cm: Confusion matrix
        output_dir: Đường dẫn lưu biểu đồ
    """
    output_dir = Path(output_dir)

    fig, ax = plt.subplots(figsize=(8, 6))

    # Vẽ heatmap
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)

    # Labels
    classes = ['Benign', 'Attack']
    ax.set(xticks=[0, 1], yticks=[0, 1],
           xticklabels=classes, yticklabels=classes,
           title='Confusion Matrix',
           ylabel='Actual',
           xlabel='Predicted')

    # Thêm text vào các ô
    thresh = cm.max() / 2.
    for i in range(2):
        for j in range(2):
            ax.text(j, i, format(cm[i, j], 'd'),
                   ha="center", va="center",
                   color="white" if cm[i, j] > thresh else "black",
                   fontsize=14)

    plt.tight_layout()
    plt.savefig(output_dir / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.close()

    print(f"📊 Đã lưu confusion matrix: {output_dir / 'confusion_matrix.png'}")


def plot_roc_curve(y_test, y_pred_prob, output_dir):
    """
    Vẽ ROC curve

    Args:
        y_test: Labels thực tế
        y_pred_prob: Xác suất dự đoán
        output_dir: Đường dẫn lưu biểu đồ
    """
    output_dir = Path(output_dir)

    # Tính ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    auc = roc_auc_score(y_test, y_pred_prob)

    fig, ax = plt.subplots(figsize=(8, 6))

    ax.plot(fpr, tpr, color='blue', linewidth=2,
            label=f'ROC curve (AUC = {auc:.4f})')
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--', linewidth=1)

    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title('Receiver Operating Characteristic (ROC) Curve',
                fontsize=14, fontweight='bold')
    ax.legend(loc="lower right")
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(output_dir / 'roc_curve.png', dpi=150, bbox_inches='tight')
    plt.close()

    print(f"📊 Đã lưu ROC curve: {output_dir / 'roc_curve.png'}")


# ============================================================================
# LƯU KẾT QUẢ
# ============================================================================

def save_results(results, history, training_time, output_dir, config):
    """
    Lưu kết quả và thông tin mô hình

    Args:
        results: Dictionary chứa metrics
        history: Training history
        training_time: Thời gian training (giây)
        output_dir: Đường dẫn lưu
        config: Config của mô hình
    """
    output_dir = Path(output_dir)

    # Tạo summary
    summary = {
        'model_name': 'CNN_1D_IDS',
        'task': 'Binary Classification - IoT Anomaly Detection',
        'dataset': 'CICIDS2018',
        'training_time_seconds': training_time,
        'training_time_minutes': training_time / 60,
        'config': config,
        'results': {
            'accuracy': float(results['accuracy']),
            'precision': float(results['precision']),
            'recall': float(results['recall']),
            'f1_score': float(results['f1_score']),
            'auc_roc': float(results['auc_roc']),
            'detection_rate': float(results['detection_rate']),
            'false_alarm_rate': float(results['false_alarm_rate']),
            'confusion_matrix': results['confusion_matrix']
        },
        'final_epoch': len(history.history['loss']),
        'best_val_loss': float(min(history.history['val_loss'])),
        'best_val_auc': float(max(history.history['val_auc'])),
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    # Lưu summary
    with open(output_dir / 'results_summary.json', 'w') as f:
        json.dump(summary, f, indent=4)

    print(f"\n💾 Đã lưu kết quả: {output_dir / 'results_summary.json'}")


# ============================================================================
# HÀM CHÍNH
# ============================================================================

def main():
    """Hàm chính để chạy toàn bộ pipeline"""

    print("\n" + "="*80)
    print("🔧 HUẤN LUYỆN MÔ HÌNH CNN - PHÁT HIỆN LƯU LƯỢNG MẠNG IOT BẤT THƯỜNG")
    print("="*80)

    # Tạo thư mục output
    output_dir = Path(OUTPUT_DIR)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Bước 1: Load dữ liệu
    X_train, X_val, X_test, y_train, y_val, y_test, metadata = load_processed_data(PROCESSED_DATA_DIR)

    # Bước 2: Xây dựng mô hình
    input_shape = (X_train.shape[1], X_train.shape[2])  # (n_features, 1)
    model = build_cnn_model(input_shape)

    # Bước 3: Huấn luyện mô hình
    history, training_time = train_model(
        model, X_train, y_train, X_val, y_val,
        output_dir, CNN_CONFIG
    )

    # Bước 4: Đánh giá mô hình
    results = evaluate_model(model, X_test, y_test)

    # Bước 5: Vẽ biểu đồ
    plot_training_history(history, output_dir)
    plot_confusion_matrix(np.array(results['confusion_matrix']), output_dir)
    plot_roc_curve(y_test, results['y_pred_prob'], output_dir)

    # Bước 6: Lưu kết quả
    save_results(results, history, training_time, output_dir, CNN_CONFIG)

    # Lưu model cuối cùng
    model.save(output_dir / 'final_model.keras')
    print(f"💾 Đã lưu model cuối cùng: {output_dir / 'final_model.keras'}")

    print("\n" + "="*80)
    print("✅ HOÀN THÀNH HUẤN LUYỆN MÔ HÌNH CNN!")
    print("="*80)
    print(f"\n📁 Tất cả kết quả được lưu tại: {output_dir}")
    print(f"   - best_model.keras: Model tốt nhất (theo val_auc)")
    print(f"   - final_model.keras: Model cuối cùng")
    print(f"   - training_history.png: Biểu đồ training")
    print(f"   - confusion_matrix.png: Confusion matrix")
    print(f"   - roc_curve.png: ROC curve")
    print(f"   - results_summary.json: Tóm tắt kết quả")

    return model, history, results


# ============================================================================
# CHẠY SCRIPT
# ============================================================================

if __name__ == "__main__":
    model, history, results = main()



In [ ]:
"""
Graph Construction for Network Traffic Data
Xây dựng đồ thị từ dữ liệu network traffic để sử dụng với GNN
Optimized for Kaggle with Garbage Collection
"""

import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os
from tqdm import tqdm
import gc  # Thêm garbage collection

# ============================================================================
# CONFIGURATION
# ============================================================================
PROCESSED_DATA_DIR = "/kaggle/input/preprocess-data/processed_data"  # Dataset đã upload
GRAPH_DATA_DIR = "/kaggle/working"  # Output directory

# Graph construction parameters
K_NEIGHBORS = 8  # Số lượng neighbors cho KNN graph
SIMILARITY_THRESHOLD = 0.5  # Ngưỡng similarity để tạo edge
GRAPH_TYPE = 'knn'  # 'knn' hoặc 'similarity'
MAX_SAMPLES = 2000000  # Giới hạn số samples để training nhanh

# ============================================================================
# GRAPH CONSTRUCTION CLASS
# ============================================================================

class NetworkTrafficGraphBuilder:
    """Class để xây dựng đồ thị từ network traffic data"""

    def __init__(self, k_neighbors=10, similarity_threshold=0.5, graph_type='knn'):
        self.k_neighbors = k_neighbors
        self.similarity_threshold = similarity_threshold
        self.graph_type = graph_type

    def build_knn_graph(self, X, batch_size=10000):
        """
        Xây dựng KNN graph từ features

        Args:
            X: Feature matrix (n_samples, n_features)
            batch_size: Batch size để xử lý (tránh memory overflow)

        Returns:
            edge_index: Edge indices (2, num_edges)
        """
        print(f"\nXây dựng KNN graph với k={self.k_neighbors}...")

        n_samples = X.shape[0]

        if n_samples <= batch_size:
            # Xử lý trực tiếp nếu dữ liệu nhỏ
            adjacency = kneighbors_graph(
                X,
                n_neighbors=self.k_neighbors,
                mode='connectivity',
                include_self=False
            )
            
            # GC sau khi tạo adjacency matrix
            gc.collect()
            
        else:
            # Xử lý theo batch nếu dữ liệu lớn - tạo edge list thay vì adjacency matrix
            print(f"  Dữ liệu lớn, xử lý theo batch ({batch_size} samples/batch)...")
            edges = []

            # Fit NearestNeighbors một lần cho toàn bộ dataset
            from sklearn.neighbors import NearestNeighbors
            print("  Fitting NearestNeighbors model...")
            nbrs = NearestNeighbors(n_neighbors=self.k_neighbors + 1, algorithm='auto').fit(X)
            
            # GC sau khi fit model
            gc.collect()

            for i in tqdm(range(0, n_samples, batch_size), desc="  Building KNN graph"):
                end_idx = min(i + batch_size, n_samples)
                batch_X = X[i:end_idx]

                # Tìm k nearest neighbors
                distances, indices = nbrs.kneighbors(batch_X)

                # Tạo edges (bỏ qua neighbor đầu tiên vì đó là chính nó)
                for local_idx in range(len(batch_X)):
                    global_idx = i + local_idx
                    for neighbor_idx in indices[local_idx][1:]:  # Skip first (itself)
                        edges.append([global_idx, neighbor_idx])

                # GC sau mỗi 10 batches
                if (i // batch_size) % 10 == 0:
                    gc.collect()

            # Chuyển edge list sang tensor
            print("  Converting edges to tensor...")
            edges = np.array(edges).T
            edge_index = torch.tensor(edges, dtype=torch.long)
            
            # GC sau khi tạo tensor
            del edges
            gc.collect()
            
            print(f"✓ Graph created: {n_samples} nodes, {edge_index.shape[1]} edges")
            return edge_index

        # Chuyển sang edge_index format (cho trường hợp không batch)
        adjacency = adjacency.tocoo()
        edge_index = torch.tensor(
            np.vstack([adjacency.row, adjacency.col]),
            dtype=torch.long
        )
        
        # GC sau khi tạo edge_index
        del adjacency
        gc.collect()

        print(f"✓ Graph created: {n_samples} nodes, {edge_index.shape[1]} edges")

        return edge_index

    def build_similarity_graph(self, X, batch_size=1000):
        """
        Xây dựng graph dựa trên cosine similarity

        Args:
            X: Feature matrix
            batch_size: Batch size

        Returns:
            edge_index: Edge indices
        """
        print(f"\nXây dựng Similarity graph (threshold={self.similarity_threshold})...")

        n_samples = X.shape[0]
        edges = []

        # Xử lý theo batch
        for i in tqdm(range(0, n_samples, batch_size), desc="  Computing similarity"):
            end_idx = min(i + batch_size, n_samples)

            # Tính similarity cho batch hiện tại với tất cả samples
            similarities = cosine_similarity(X[i:end_idx], X)

            # Tìm các edges có similarity > threshold
            for local_idx in range(end_idx - i):
                global_idx = i + local_idx
                # Lấy indices có similarity > threshold (không bao gồm chính nó)
                similar_indices = np.where(
                    (similarities[local_idx] > self.similarity_threshold) &
                    (np.arange(n_samples) != global_idx)
                )[0]

                # Thêm edges
                for j in similar_indices:
                    edges.append([global_idx, j])

            # GC sau mỗi batch để giải phóng similarity matrix
            del similarities
            if i % (batch_size * 10) == 0:
                gc.collect()

        if len(edges) == 0:
            print("  ⚠ Không tìm thấy edges, giảm threshold hoặc dùng KNN graph")
            # Fallback to KNN
            return self.build_knn_graph(X, batch_size)

        # Chuyển sang tensor
        print("  Converting edges to tensor...")
        edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
        
        # GC sau khi tạo tensor
        del edges
        gc.collect()

        print(f"✓ Graph created: {n_samples} nodes, {edge_index.shape[1]} edges")

        return edge_index

    def create_graph_data(self, X, y, edge_index):
        """
        Tạo PyTorch Geometric Data object

        Args:
            X: Node features
            y: Node labels
            edge_index: Edge indices

        Returns:
            Data object
        """
        # Chuyển đổi sang tensor
        x = torch.tensor(X, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.long)

        # Tạo Data object
        data = Data(x=x, edge_index=edge_index, y=y)
        
        # GC sau khi tạo Data object
        gc.collect()

        return data


class CICIDSGraphDataset(InMemoryDataset):
    """Custom Dataset cho CICIDS2018 Graph Data"""

    def __init__(self, root, data_list=None, transform=None, pre_transform=None):
        self.data_list = data_list
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def process(self):
        if self.data_list is not None:
            data_list = self.data_list

            if self.pre_filter is not None:
                data_list = [d for d in data_list if self.pre_filter(d)]

            if self.pre_transform is not None:
                data_list = [self.pre_transform(d) for d in data_list]

            data, slices = self.collate(data_list)
            torch.save((data, slices), self.processed_paths[0])


# ============================================================================
# MAIN FUNCTION
# ============================================================================

def build_graph_dataset():
    """Xây dựng graph dataset từ processed data"""

    print("=" * 80)
    print("BUILDING GRAPH DATASET FOR GNN")
    print("=" * 80)

    # Tạo output directory
    os.makedirs(GRAPH_DATA_DIR, exist_ok=True)

    # Load processed data
    print("\nLoading processed data...")
    X = np.load(os.path.join(PROCESSED_DATA_DIR, "X_features.npy"))
    y_binary = np.load(os.path.join(PROCESSED_DATA_DIR, "y_binary.npy"))
    y_multi = np.load(os.path.join(PROCESSED_DATA_DIR, "y_multi.npy"))

    with open(os.path.join(PROCESSED_DATA_DIR, "metadata.pkl"), 'rb') as f:
        metadata = pickle.load(f)

    print(f"✓ Loaded data: {X.shape[0]:,} samples, {X.shape[1]} features")
    print(f"✓ Binary classes: {len(np.unique(y_binary))}")
    print(f"✓ Multi classes: {len(np.unique(y_multi))}")
    
    # GC sau khi load data
    gc.collect()

    # Giảm kích thước nếu quá lớn (optional - để training nhanh hơn)
    if X.shape[0] > MAX_SAMPLES:
        print(f"\n⚠ Dataset lớn ({X.shape[0]:,} samples), sampling {MAX_SAMPLES:,} samples...")
        indices = np.random.choice(X.shape[0], MAX_SAMPLES, replace=False)
        X = X[indices]
        y_binary = y_binary[indices]
        y_multi = y_multi[indices]
        print(f"✓ Sampled data: {X.shape[0]:,} samples")
        
        # GC sau khi sampling
        del indices
        gc.collect()

    # Build graph
    builder = NetworkTrafficGraphBuilder(
        k_neighbors=K_NEIGHBORS,
        similarity_threshold=SIMILARITY_THRESHOLD,
        graph_type=GRAPH_TYPE
    )

    if GRAPH_TYPE == 'knn':
        edge_index = builder.build_knn_graph(X)
    else:
        edge_index = builder.build_similarity_graph(X)
    
    # GC sau khi build graph
    gc.collect()

    # Create graph data objects
    print("\nCreating graph data objects...")

    # Binary classification graph
    print("  Creating binary classification graph...")
    graph_binary = builder.create_graph_data(X, y_binary, edge_index)
    
    # GC sau khi tạo binary graph
    gc.collect()

    # Multi-class classification graph
    print("  Creating multi-class classification graph...")
    graph_multi = builder.create_graph_data(X, y_multi, edge_index)
    
    # GC sau khi tạo multi graph
    gc.collect()

    # Save graphs
    print("\nSaving graph data...")
    torch.save(graph_binary, os.path.join(GRAPH_DATA_DIR, "graph_binary.pt"))
    torch.save(graph_multi, os.path.join(GRAPH_DATA_DIR, "graph_multi.pt"))

    # Save edge_index separately
    torch.save(edge_index, os.path.join(GRAPH_DATA_DIR, "edge_index.pt"))

    # Save graph metadata
    graph_metadata = {
        'n_nodes': X.shape[0],
        'n_features': X.shape[1],
        'n_edges': edge_index.shape[1],
        'k_neighbors': K_NEIGHBORS,
        'graph_type': GRAPH_TYPE,
        'avg_degree': edge_index.shape[1] / X.shape[0]
    }

    with open(os.path.join(GRAPH_DATA_DIR, "graph_metadata.pkl"), 'wb') as f:
        pickle.dump(graph_metadata, f)
    
    # GC cuối cùng
    gc.collect()

    print("\n" + "=" * 80)
    print("GRAPH CONSTRUCTION COMPLETED!")
    print("=" * 80)
    print(f"Nodes: {graph_metadata['n_nodes']:,}")
    print(f"Features per node: {graph_metadata['n_features']}")
    print(f"Edges: {graph_metadata['n_edges']:,}")
    print(f"Average degree: {graph_metadata['avg_degree']:.2f}")
    print(f"Output directory: {GRAPH_DATA_DIR}")
    print("=" * 80)

    return graph_binary, graph_multi, graph_metadata


if __name__ == "__main__":
    graph_binary, graph_multi, metadata = build_graph_dataset()

In [ ]:
"""
GNN Training Script for Kaggle - IoT Network Anomaly Detection
Script hoàn chỉnh bao gồm cả model definitions và training
Chỉ cần chạy file này trên Kaggle
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, BatchNorm
import numpy as np
import pickle
import os
import shutil
from datetime import datetime
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# GNN MODEL DEFINITIONS
# ============================================================================

class GCN(nn.Module):
    """Graph Convolutional Network"""

    def __init__(self, in_channels, hidden_channels, num_classes, num_layers=3, dropout=0.5):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # First layer
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Output layer
        self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Classifier
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


class GAT(nn.Module):
    """Graph Attention Network"""

    def __init__(self, in_channels, hidden_channels, num_classes,
                 num_layers=3, heads=4, dropout=0.5):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # First layer
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout))
        self.batch_norms.append(BatchNorm(hidden_channels * heads))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels * heads, hidden_channels,
                                     heads=heads, dropout=dropout))
            self.batch_norms.append(BatchNorm(hidden_channels * heads))

        # Output layer
        self.convs.append(GATConv(hidden_channels * heads, hidden_channels,
                                 heads=1, concat=False, dropout=dropout))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Classifier
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


class GraphSAGE(nn.Module):
    """GraphSAGE"""

    def __init__(self, in_channels, hidden_channels, num_classes,
                 num_layers=3, dropout=0.5, aggregator='mean'):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # First layer
        self.convs.append(SAGEConv(in_channels, hidden_channels, aggr=aggregator))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr=aggregator))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Output layer
        self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr=aggregator))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Classifier
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


class HybridGNN(nn.Module):
    """Hybrid GNN combining GCN and GAT"""

    def __init__(self, in_channels, hidden_channels, num_classes,
                 num_layers=3, heads=4, dropout=0.5):
        super(HybridGNN, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        # GCN branch
        self.gcn_convs = nn.ModuleList()
        self.gcn_bns = nn.ModuleList()

        # GAT branch
        self.gat_convs = nn.ModuleList()
        self.gat_bns = nn.ModuleList()

        # First layer
        self.gcn_convs.append(GCNConv(in_channels, hidden_channels))
        self.gcn_bns.append(BatchNorm(hidden_channels))
        self.gat_convs.append(GATConv(in_channels, hidden_channels // heads, heads=heads))
        self.gat_bns.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.gcn_convs.append(GCNConv(hidden_channels, hidden_channels))
            self.gcn_bns.append(BatchNorm(hidden_channels))
            self.gat_convs.append(GATConv(hidden_channels, hidden_channels // heads, heads=heads))
            self.gat_bns.append(BatchNorm(hidden_channels))

        # Fusion
        self.fusion = nn.Linear(hidden_channels * 2, hidden_channels)
        self.fusion_bn = BatchNorm(hidden_channels)

        # Classifier
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        # GCN branch
        x_gcn = x
        for i in range(self.num_layers - 1):
            x_gcn = self.gcn_convs[i](x_gcn, edge_index)
            x_gcn = self.gcn_bns[i](x_gcn)
            x_gcn = F.relu(x_gcn)
            x_gcn = F.dropout(x_gcn, p=self.dropout, training=self.training)

        # GAT branch
        x_gat = x
        for i in range(self.num_layers - 1):
            x_gat = self.gat_convs[i](x_gat, edge_index)
            x_gat = self.gat_bns[i](x_gat)
            x_gat = F.elu(x_gat)
            x_gat = F.dropout(x_gat, p=self.dropout, training=self.training)

        # Fusion
        x = torch.cat([x_gcn, x_gat], dim=1)
        x = self.fusion(x)
        x = self.fusion_bn(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


def create_model(model_name, in_channels, hidden_channels, num_classes, **kwargs):
    """Factory function để tạo model"""
    models = {
        'GCN': GCN,
        'GAT': GAT,
        'GraphSAGE': GraphSAGE,
        'Hybrid': HybridGNN
    }
    if model_name not in models:
        raise ValueError(f"Model {model_name} không hỗ trợ. Chọn: {list(models.keys())}")
    return models[model_name](in_channels, hidden_channels, num_classes, **kwargs)


def count_parameters(model):
    """Đếm số parameters"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# ============================================================================
# KAGGLE CONFIGURATION
# ============================================================================
WORKING_DIR = "/kaggle/working"
GRAPH_DATA_DIR = "/kaggle/working"
MODEL_DIR = os.path.join(WORKING_DIR, "models")
RESULTS_DIR = os.path.join(WORKING_DIR, "results")

# Model config
MODEL_NAME = 'GAT'  # 'GCN', 'GAT', 'GraphSAGE', 'Hybrid'
HIDDEN_CHANNELS = 128
NUM_LAYERS = 3
HEADS = 4
DROPOUT = 0.3

# Training config
LEARNING_RATE = 0.001
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 30
PATIENCE = 15
TASK = 'binary'  # 'binary' hoặc 'multi'

# Data split
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using device: {DEVICE}")

# ============================================================================
# TRAINING CLASS
# ============================================================================

class GNNTrainer:
    """Class để train GNN model"""

    def __init__(self, model, device, task='binary'):
        self.model = model.to(device)
        self.device = device
        self.task = task
        self.history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'learning_rate': []
        }
        self.best_val_acc = 0
        self.best_epoch = 0

    def train_epoch(self, data, train_mask, optimizer):
        """Train một epoch"""
        self.model.train()
        optimizer.zero_grad()

        out = self.model(data.x, data.edge_index)
        loss = F.cross_entropy(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        pred = out[train_mask].argmax(dim=1)
        acc = (pred == data.y[train_mask]).float().mean()
        return loss.item(), acc.item()

    @torch.no_grad()
    def evaluate(self, data, mask):
        """Evaluate model"""
        self.model.eval()
        out = self.model(data.x, data.edge_index)
        loss = F.cross_entropy(out[mask], data.y[mask])
        pred = out[mask].argmax(dim=1)
        acc = (pred == data.y[mask]).float().mean()
        return loss.item(), acc.item(), pred.cpu().numpy(), data.y[mask].cpu().numpy()

    def train(self, data, train_mask, val_mask, optimizer, scheduler, num_epochs, patience):
        """Full training loop"""
        print("\n" + "=" * 80)
        print("🚀 TRAINING GNN MODEL")
        print("=" * 80)
        print(f"📍 Device: {self.device}")
        print(f"🧠 Model: {self.model.__class__.__name__}")
        print(f"📊 Parameters: {count_parameters(self.model):,}")
        print(f"⏰ Epochs: {num_epochs}")
        print("=" * 80 + "\n")

        patience_counter = 0

        for epoch in range(1, num_epochs + 1):
            train_loss, train_acc = self.train_epoch(data, train_mask, optimizer)
            val_loss, val_acc, _, _ = self.evaluate(data, val_mask)

            if scheduler is not None:
                scheduler.step(val_loss)

            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['learning_rate'].append(optimizer.param_groups[0]['lr'])

            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_epoch = epoch
                patience_counter = 0
                self.save_model(os.path.join(MODEL_DIR, f'best_model_{self.task}.pt'))
            else:
                patience_counter += 1

            if epoch % 10 == 0 or epoch == 1:
                print(f"Epoch {epoch:3d}/{num_epochs} | "
                      f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                      f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            if patience_counter >= patience:
                print(f"\n⏹️  Early stopping at epoch {epoch}")
                break

        print(f"\n✅ Best Val Acc: {self.best_val_acc:.4f} at epoch {self.best_epoch}")
        return self.history

    def test(self, data, test_mask):
        """Test model"""
        print("\n" + "=" * 80)
        print("🧪 TESTING MODEL")
        print("=" * 80)

        test_loss, test_acc, pred, true = self.evaluate(data, test_mask)

        print(f"📊 Test Accuracy: {test_acc:.4f}")

        precision = precision_score(true, pred, average='weighted', zero_division=0)
        recall = recall_score(true, pred, average='weighted', zero_division=0)
        f1 = f1_score(true, pred, average='weighted', zero_division=0)

        print(f"📊 Precision: {precision:.4f}")
        print(f"📊 Recall: {recall:.4f}")
        print(f"📊 F1-Score: {f1:.4f}")

        # ROC-AUC for binary
        roc_auc = None
        if self.task == 'binary':
            try:
                self.model.eval()
                with torch.no_grad():
                    out = self.model(data.x, data.edge_index)
                    probs = F.softmax(out[test_mask], dim=1)[:, 1].cpu().numpy()
                roc_auc = roc_auc_score(true, probs)
                print(f"📊 ROC-AUC: {roc_auc:.4f}")
            except:
                pass

        cm = confusion_matrix(true, pred)
        print("\n" + "-" * 80)
        print("📋 Classification Report:")
        print("-" * 80)
        print(classification_report(true, pred, zero_division=0))

        return {
            'test_loss': test_loss,
            'test_acc': test_acc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'roc_auc': roc_auc,
            'confusion_matrix': cm,
            'predictions': pred,
            'true_labels': true
        }

    def save_model(self, path):
        """Save model"""
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'best_val_acc': self.best_val_acc,
            'best_epoch': self.best_epoch,
            'history': self.history
        }, path)

    def load_model(self, path):
        """Load model"""
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.best_val_acc = checkpoint['best_val_acc']
        self.best_epoch = checkpoint['best_epoch']
        self.history = checkpoint['history']


# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_training_history(history, save_path=None):
    """Plot training history"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_confusion_matrix(cm, class_names=None, save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# ============================================================================
# MAIN
# ============================================================================

def main():
    """Main training function"""
    
    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(RESULTS_DIR, exist_ok=True)

    print("\n" + "=" * 80)
    print("🤖 GNN TRAINING FOR IoT ANOMALY DETECTION - KAGGLE")
    print("=" * 80 + "\n")

    # Load graph
    print("📂 Loading graph data...")
    graph_file = f"graph_{TASK}.pt"
    graph_path = os.path.join(GRAPH_DATA_DIR, graph_file)
    
    if not os.path.exists(graph_path):
        print(f"❌ ERROR: Graph file not found: {graph_path}")
        print("📁 Available files:")
        for f in os.listdir(WORKING_DIR):
            print(f"  - {f}")
        return
    
    data = torch.load(graph_path, weights_only=False)
    data = data.to(DEVICE)

    print(f"✅ Graph: {data.num_nodes:,} nodes, {data.num_edges:,} edges")
    print(f"✅ Features: {data.num_features}")
    print(f"✅ Classes: {len(torch.unique(data.y))}")

    # Load metadata
    metadata_path = os.path.join(GRAPH_DATA_DIR, "graph_metadata.pkl")
    class_names = None
    if os.path.exists(metadata_path):
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
            if TASK in metadata and 'class_names' in metadata[TASK]:
                class_names = metadata[TASK]['class_names']
    
    if class_names is None:
        class_names = ['Benign', 'Attack'] if TASK == 'binary' else [f'Class_{i}' for i in range(len(torch.unique(data.y)))]

    # Create masks
    print("\n📊 Creating data splits...")
    num_nodes = data.num_nodes
    indices = torch.randperm(num_nodes)

    train_size = int(num_nodes * TRAIN_RATIO)
    val_size = int(num_nodes * VAL_RATIO)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    print(f"✅ Train: {train_mask.sum():,} | Val: {val_mask.sum():,} | Test: {test_mask.sum():,}")

    # Create model
    print(f"\n🏗️  Creating {MODEL_NAME} model...")
    num_classes = len(torch.unique(data.y))

    model_kwargs = {'num_layers': NUM_LAYERS, 'dropout': DROPOUT}
    if MODEL_NAME in ['GAT', 'Hybrid']:
        model_kwargs['heads'] = HEADS

    model = create_model(
        MODEL_NAME,
        in_channels=data.num_features,
        hidden_channels=HIDDEN_CHANNELS,
        num_classes=num_classes,
        **model_kwargs
    )

    print(f"✅ Model: {count_parameters(model):,} parameters")

    # Train
    trainer = GNNTrainer(model, DEVICE, task=TASK)
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

    history = trainer.train(data, train_mask, val_mask, optimizer, scheduler, NUM_EPOCHS, PATIENCE)

    # Test
    print("\n🔄 Loading best model...")
    trainer.load_model(os.path.join(MODEL_DIR, f'best_model_{TASK}.pt'))
    results = trainer.test(data, test_mask)

    # Save results
    print("\n💾 Saving results...")
    plot_training_history(history, os.path.join(RESULTS_DIR, f'training_history_{TASK}.png'))
    plot_confusion_matrix(results['confusion_matrix'], class_names, os.path.join(RESULTS_DIR, f'confusion_matrix_{TASK}.png'))

    with open(os.path.join(RESULTS_DIR, f'results_{TASK}.pkl'), 'wb') as f:
        pickle.dump(results, f)

    config = {
        'model_name': MODEL_NAME,
        'hidden_channels': HIDDEN_CHANNELS,
        'num_layers': NUM_LAYERS,
        'dropout': DROPOUT,
        'best_epoch': trainer.best_epoch,
        'best_val_acc': trainer.best_val_acc,
        'test_results': {
            'accuracy': results['test_acc'],
            'precision': results['precision'],
            'recall': results['recall'],
            'f1': results['f1'],
            'roc_auc': results['roc_auc']
        }
    }

    with open(os.path.join(RESULTS_DIR, f'config_{TASK}.pkl'), 'wb') as f:
        pickle.dump(config, f)

    # Summary
    with open(os.path.join(RESULTS_DIR, 'summary.txt'), 'w') as f:
        f.write("=" * 80 + "\n")
        f.write("TRAINING SUMMARY\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Model: {MODEL_NAME}\n")
        f.write(f"Task: {TASK}\n")
        f.write(f"Best Epoch: {trainer.best_epoch}\n")
        f.write(f"Best Val Acc: {trainer.best_val_acc:.4f}\n\n")
        f.write(f"Test Results:\n")
        f.write(f"  Accuracy: {results['test_acc']:.4f}\n")
        f.write(f"  Precision: {results['precision']:.4f}\n")
        f.write(f"  Recall: {results['recall']:.4f}\n")
        f.write(f"  F1-Score: {results['f1']:.4f}\n")
        if results['roc_auc']:
            f.write(f"  ROC-AUC: {results['roc_auc']:.4f}\n")

    # ZIP for download
    print("\n" + "=" * 80)
    print("📦 CREATING ZIP FILES")
    print("=" * 80)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_results = f'gnn_results_{TASK}_{timestamp}'
    zip_models = f'gnn_models_{TASK}_{timestamp}'

    shutil.make_archive(os.path.join(WORKING_DIR, zip_results), 'zip', RESULTS_DIR)
    shutil.make_archive(os.path.join(WORKING_DIR, zip_models), 'zip', MODEL_DIR)

    print(f"✅ Results: {zip_results}.zip")
    print(f"✅ Models: {zip_models}.zip")

    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETED!")
    print("=" * 80)
    print(f"📊 Test Accuracy: {results['test_acc']:.4f}")
    print(f"📊 F1-Score: {results['f1']:.4f}")
    print(f"\n📥 Download: {zip_results}.zip & {zip_models}.zip")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()

In [ ]:
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

In [ ]:
# Install PyTorch Geometric
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
!pip install torch-geometric

# Upgrade scikit-learn (nếu cần)
!pip install --upgrade scikit-learn

# Install tqdm
!pip install tqdm

In [ ]:
"""
GNN Training Script - FIXED VERSION with Mini-Batch Support
Key fixes:
1. Added NeighborLoader for mini-batch training
2. Reduced memory footprint significantly
3. Added gradient accumulation option
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, BatchNorm
# Removed: from torch_geometric.loader import NeighborLoader
import numpy as np
import pickle
import os
import shutil
from datetime import datetime
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# MODEL DEFINITIONS (Same as before - these are fine)
# ============================================================================

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes, num_layers=3, dropout=0.5):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes,
                 num_layers=3, heads=4, dropout=0.5):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout))
        self.batch_norms.append(BatchNorm(hidden_channels * heads))

        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels * heads, hidden_channels,
                                     heads=heads, dropout=dropout))
            self.batch_norms.append(BatchNorm(hidden_channels * heads))

        self.convs.append(GATConv(hidden_channels * heads, hidden_channels,
                                 heads=1, concat=False, dropout=dropout))
        self.batch_norms.append(BatchNorm(hidden_channels))
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes,
                 num_layers=3, dropout=0.5, aggregator='mean'):
        super(GraphSAGE, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        self.convs.append(SAGEConv(in_channels, hidden_channels, aggr=aggregator))
        self.batch_norms.append(BatchNorm(hidden_channels))

        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr=aggregator))
            self.batch_norms.append(BatchNorm(hidden_channels))

        self.convs.append(SAGEConv(hidden_channels, hidden_channels, aggr=aggregator))
        self.batch_norms.append(BatchNorm(hidden_channels))
        self.classifier = nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = self.batch_norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.classifier(x)
        return x


def create_model(model_name, in_channels, hidden_channels, num_classes, **kwargs):
    models = {'GCN': GCN, 'GAT': GAT, 'GraphSAGE': GraphSAGE}
    if model_name not in models:
        raise ValueError(f"Model {model_name} not supported. Choose: {list(models.keys())}")
    return models[model_name](in_channels, hidden_channels, num_classes, **kwargs)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# ============================================================================
# CONFIGURATION - OPTIMIZED FOR MEMORY
# ============================================================================
WORKING_DIR = "/kaggle/working"
GRAPH_DATA_DIR = "/kaggle/input/model/pytorch/default/1"
MODEL_DIR = os.path.join(WORKING_DIR, "models")
RESULTS_DIR = os.path.join(WORKING_DIR, "results")

# Model config - Reduced for memory
MODEL_NAME = 'GraphSAGE'  # GraphSAGE is most memory-efficient
HIDDEN_CHANNELS = 64      # Reduced from 128
NUM_LAYERS = 2            # Reduced from 3
HEADS = 2                 # Reduced from 4 (for GAT)
DROPOUT = 0.3

# Training config
LEARNING_RATE = 0.001
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 30
PATIENCE = 15
TASK = 'binary'

# MINI-BATCH CONFIG
BATCH_SIZE = 2048         # Process nodes in batches
NUM_WORKERS = 0           # Not used in simple batching

# Gradient accumulation (simulate larger batch)
ACCUMULATION_STEPS = 4

# Data split
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using device: {DEVICE}")

# ============================================================================
# MINI-BATCH TRAINER CLASS - COMPLETELY REWRITTEN
# ============================================================================

class MiniBatchGNNTrainer:
    """Memory-efficient mini-batch trainer"""

    def __init__(self, model, device, task='binary'):
        self.model = model.to(device)
        self.device = device
        self.task = task
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [], 'learning_rate': []
        }
        self.best_val_acc = 0
        self.best_epoch = 0

    def train_epoch(self, train_batches, data, optimizer, accumulation_steps=1):
        """Train one epoch with simple batches"""
        self.model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        
        optimizer.zero_grad()
        
        # Move full graph to device once
        data = data.to(self.device)
        
        for i, batch_idx in enumerate(train_batches):
            batch_idx = batch_idx.to(self.device)
            
            # Forward pass on full graph
            out = self.model(data.x, data.edge_index)
            
            # Compute loss only on batch nodes
            loss = F.cross_entropy(out[batch_idx], data.y[batch_idx])
            loss = loss / accumulation_steps
            
            # Backward pass
            loss.backward()
            
            # Update weights
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            # Metrics
            with torch.no_grad():
                pred = out[batch_idx].argmax(dim=1)
                total_correct += (pred == data.y[batch_idx]).sum().item()
                total_loss += loss.item() * accumulation_steps * len(batch_idx)
                total_samples += len(batch_idx)
        
        # Final update
        if (i + 1) % accumulation_steps != 0:
            optimizer.step()
            optimizer.zero_grad()
        
        return total_loss / total_samples, total_correct / total_samples

    @torch.no_grad()
    def evaluate(self, batches, data):
        """Evaluate with simple batches"""
        self.model.eval()
        total_loss = 0
        total_correct = 0
        total_samples = 0
        all_preds = []
        all_labels = []
        
        data = data.to(self.device)
        
        for batch_idx in batches:
            batch_idx = batch_idx.to(self.device)
            
            # Forward on full graph
            out = self.model(data.x, data.edge_index)
            
            # Loss and predictions on batch
            loss = F.cross_entropy(out[batch_idx], data.y[batch_idx])
            pred = out[batch_idx].argmax(dim=1)
            
            total_correct += (pred == data.y[batch_idx]).sum().item()
            total_loss += loss.item() * len(batch_idx)
            total_samples += len(batch_idx)
            
            all_preds.append(pred.cpu())
            all_labels.append(data.y[batch_idx].cpu())
        
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()
        
        return total_loss / total_samples, total_correct / total_samples, all_preds, all_labels

    def train(self, train_batches, val_batches, data, optimizer, scheduler, num_epochs, patience):
        """Full training loop"""
        print("\n" + "=" * 80)
        print("🚀 TRAINING GNN MODEL (BATCHED)")
        print("=" * 80)
        print(f"📍 Device: {self.device}")
        print(f"🧠 Model: {self.model.__class__.__name__}")
        print(f"📊 Parameters: {count_parameters(self.model):,}")
        print(f"🎯 Batch size: {BATCH_SIZE}")
        print(f"⏰ Epochs: {num_epochs}")
        print("=" * 80 + "\n")

        patience_counter = 0

        for epoch in range(1, num_epochs + 1):
            train_loss, train_acc = self.train_epoch(train_batches, data, optimizer, ACCUMULATION_STEPS)
            val_loss, val_acc, _, _ = self.evaluate(val_batches, data)

            if scheduler is not None:
                scheduler.step(val_loss)

            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['learning_rate'].append(optimizer.param_groups[0]['lr'])

            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_epoch = epoch
                patience_counter = 0
                self.save_model(os.path.join(MODEL_DIR, f'best_model_{self.task}.pt'))
            else:
                patience_counter += 1

            if epoch % 5 == 0 or epoch == 1:
                print(f"Epoch {epoch:3d}/{num_epochs} | "
                      f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
                      f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            if patience_counter >= patience:
                print(f"\n⏹️  Early stopping at epoch {epoch}")
                break

        print(f"\n✅ Best Val Acc: {self.best_val_acc:.4f} at epoch {self.best_epoch}")
        return self.history

    def test(self, test_batches, data):
        """Test model"""
        print("\n" + "=" * 80)
        print("🧪 TESTING MODEL")
        print("=" * 80)

        test_loss, test_acc, pred, true = self.evaluate(test_batches, data)

        print(f"📊 Test Accuracy: {test_acc:.4f}")

        precision = precision_score(true, pred, average='weighted', zero_division=0)
        recall = recall_score(true, pred, average='weighted', zero_division=0)
        f1 = f1_score(true, pred, average='weighted', zero_division=0)

        print(f"📊 Precision: {precision:.4f}")
        print(f"📊 Recall: {recall:.4f}")
        print(f"📊 F1-Score: {f1:.4f}")

        cm = confusion_matrix(true, pred)
        print("\n" + "-" * 80)
        print("📋 Classification Report:")
        print("-" * 80)
        print(classification_report(true, pred, zero_division=0))

        return {
            'test_loss': test_loss, 'test_acc': test_acc,
            'precision': precision, 'recall': recall, 'f1': f1,
            'confusion_matrix': cm, 'predictions': pred, 'true_labels': true
        }

    def save_model(self, path):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'best_val_acc': self.best_val_acc,
            'best_epoch': self.best_epoch,
            'history': self.history
        }, path)

    def load_model(self, path):
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.best_val_acc = checkpoint['best_val_acc']
        self.best_epoch = checkpoint['best_epoch']
        self.history = checkpoint['history']


# ============================================================================
# VISUALIZATION (Same as before)
# ============================================================================

def plot_training_history(history, save_path=None):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_confusion_matrix(cm, class_names=None, save_path=None):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# ============================================================================
# MAIN - REWRITTEN WITH MINI-BATCH LOADERS
# ============================================================================

def main():
    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(RESULTS_DIR, exist_ok=True)

    print("\n" + "=" * 80)
    print("🤖 GNN TRAINING (MEMORY-OPTIMIZED)")
    print("=" * 80 + "\n")

    # Load graph
    print("📂 Loading graph data...")
    graph_file = f"graph_{TASK}.pt"
    graph_path = os.path.join(GRAPH_DATA_DIR, graph_file)
    
    if not os.path.exists(graph_path):
        print(f"❌ ERROR: Graph file not found: {graph_path}")
        return
    
    data = torch.load(graph_path, weights_only=False)

    print(f"✅ Graph: {data.num_nodes:,} nodes, {data.num_edges:,} edges")

    # Create masks
    print("\n📊 Creating data splits...")
    num_nodes = data.num_nodes
    indices = torch.randperm(num_nodes)

    train_size = int(num_nodes * TRAIN_RATIO)
    val_size = int(num_nodes * VAL_RATIO)

    train_idx = indices[:train_size]
    val_idx = indices[train_size:train_size + val_size]
    test_idx = indices[train_size + val_size:]

    print(f"✅ Train: {len(train_idx):,} | Val: {len(val_idx):,} | Test: {len(test_idx):,}")

    # Create simple batched indices (no neighbor sampling)
    print(f"\n🔧 Creating simple batched processing...")
    
    def create_simple_batches(indices, batch_size):
        """Create simple batches without neighbor sampling"""
        return [indices[i:i+batch_size] for i in range(0, len(indices), batch_size)]
    
    train_batches = create_simple_batches(train_idx, BATCH_SIZE)
    val_batches = create_simple_batches(val_idx, BATCH_SIZE)
    test_batches = create_simple_batches(test_idx, BATCH_SIZE)
    
    print(f"✅ Train batches: {len(train_batches)}")
    print(f"✅ Val batches: {len(val_batches)}")
    print(f"✅ Test batches: {len(test_batches)}")

    # Create model
    print(f"\n🏗️  Creating {MODEL_NAME} model...")
    num_classes = len(torch.unique(data.y))

    model_kwargs = {'num_layers': NUM_LAYERS, 'dropout': DROPOUT}
    if MODEL_NAME == 'GAT':
        model_kwargs['heads'] = HEADS

    model = create_model(
        MODEL_NAME,
        in_channels=data.num_features,
        hidden_channels=HIDDEN_CHANNELS,
        num_classes=num_classes,
        **model_kwargs
    )

    print(f"✅ Model: {count_parameters(model):,} parameters")

    # Train
    trainer = MiniBatchGNNTrainer(model, DEVICE, task=TASK)
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

    history = trainer.train(train_batches, val_batches, data, optimizer, scheduler, NUM_EPOCHS, PATIENCE)

    # Test
    print("\n🔄 Loading best model...")
    trainer.load_model(os.path.join(MODEL_DIR, f'best_model_{TASK}.pt'))
    results = trainer.test(test_batches, data)

    # Save results
    print("\n💾 Saving results...")
    plot_training_history(history, os.path.join(RESULTS_DIR, f'training_history_{TASK}.png'))
    plot_confusion_matrix(results['confusion_matrix'], ['Benign', 'Attack'], 
                          os.path.join(RESULTS_DIR, f'confusion_matrix_{TASK}.png'))

    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETED!")
    print("=" * 80)
    print(f"📊 Test Accuracy: {results['test_acc']:.4f}")
    print(f"📊 F1-Score: {results['f1']:.4f}")
    print("=" * 80 + "\n")


if __name__ == "__main__":
    main()

In [ ]:
!pip install -q torch-sparse